## Импорт библиотек

In [ ]:
import os
import subprocess
import gc
import pickle
import time

import numpy as np
import pandas as pd
import polars as pl

from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.metrics import mean_absolute_error
from sklearn.compose import TransformedTargetRegressor
from sklearn.ensemble import VotingRegressor

from joblib import dump
from joblib import load

import lightgbm as lgb
import catboost as cb

import optuna

import matplotlib.pyplot as plt

#from mlxtend.evaluate.time_series import GroupTimeSeriesSplit, plot_splits

In [ ]:
# Время старта работы ноутбука
notebook_starttime = time.time()

In [ ]:
# Возвращает сколько уже работает ноутбук
def p_time():
    #
    run_time = round(time.time() - notebook_starttime)
    return str(run_time).zfill(5)+' sec:'

## Настройка: сабмит или локально

In [ ]:
# Ставим is_local в True, если локально работаем, если сабмитим - ставим в False
#is_local = False
is_local = True

# Ставим is_gpu в True, если будем работать на GPU, если на процессоре - ставим в False
# is_gpu = False
is_gpu = True

# Ставим is_tuning в True, если запускаем подбор гиперпараметров в Optuna
is_tuning = False

# Начальная дата обучения модели
training_start_date = 'datetime >= "2022-01-01 00:00:00"'

# Устанавливаем время начало переобучения. Начинаем переобучать модели заново,
# когда подходит дата предсказаний, которая идет в скор.
scor_start_time = pd.to_datetime('2023-06-01')

# Ставим в False, если не хотитим отключать контроль времени выполнения ноутбука
# (не обучать модели заново после 8 часов 30 минут)
# Если хотим, чтобы модели обучались заново и после лимита, ставим True
# is_disable_run_time_limit = True
is_disable_run_time_limit = False

In [ ]:
class MonthlyKFold:
    def __init__(self, n_splits=3):
        self.n_splits = n_splits
        
    def split(self, X, y, groups=None):
        dates = 12 * X["year"] + X["month"]
        timesteps = sorted(dates.unique().tolist())
        X = X.reset_index()
        
        for t in timesteps[-self.n_splits:]:
            idx_train = X[dates.values < t].index
            idx_test = X[dates.values == t].index
            
            yield idx_train, idx_test
            
    def get_n_splits(self, X, y, groups=None):
        return self.n_splits

In [ ]:
if is_tuning:
    cv = MonthlyKFold()
    CV = GroupTimeSeriesSplit(test_size=3, n_splits=3, window_type='rolling', shift_size=2)

## Feature Engineering sub

In [ ]:
def feature_eng(df_data, df_client, df_gas, df_electricity, df_forecast, df_historical, df_location, df_target, working_days):
    working_days = (
        working_days
        .with_columns(
            pl.col("date").cast(pl.Date)
        )
    )
    
    df_data = (
        df_data
        .with_columns(
            pl.col("datetime").cast(pl.Date).alias("date"),
        )
    )
    
    df_client = (
        df_client
        .with_columns(
            (pl.col("date") + pl.duration(days=2)).cast(pl.Date)
        )
    )
    
    df_gas = (
        df_gas
        .rename({"forecast_date": "date"})
        .with_columns(
            (pl.col("date") + pl.duration(days=1)).cast(pl.Date)
        )
    )
    
    df_electricity = (
        df_electricity
        .rename({"forecast_date": "datetime"})
        .with_columns(
            pl.col("datetime") + pl.duration(days=1)
        )
    )
    
    df_location = (
        df_location
        .with_columns(
            pl.col("latitude").cast(pl.datatypes.Float32),
            pl.col("longitude").cast(pl.datatypes.Float32)
        )
    )
    
    df_forecast = (
        df_forecast
        .rename({"forecast_datetime": "datetime"})
        .with_columns(
            pl.col("latitude").cast(pl.datatypes.Float32),
            pl.col("longitude").cast(pl.datatypes.Float32),
            #pl.col('datetime').dt.convert_time_zone("Europe/Bucharest").dt.replace_time_zone(None).cast(pl.Datetime("us")),
            pl.col('datetime').dt.replace_time_zone(None).cast(pl.Datetime("us"))
            #pl.col('datetime').cast(pl.Datetime)
        )
        .join(df_location, how="left", on=["longitude", "latitude"])
        .drop("longitude", "latitude")
    )
    
    df_historical = (
        df_historical
        .with_columns(
            pl.col("latitude").cast(pl.datatypes.Float32),
            pl.col("longitude").cast(pl.datatypes.Float32),
            pl.col("datetime") + pl.duration(hours=37)
        )
        .join(df_location, how="left", on=["longitude", "latitude"])
        .drop("longitude", "latitude")
    )
    
    df_forecast_date = (
        df_forecast
        .group_by("datetime").mean()
        .drop("county")
    )
    
    df_forecast_local = (
        df_forecast
        .filter(pl.col("county").is_not_null())
        .group_by("county", "datetime").mean()
    )
    
    df_historical_date = (
        df_historical
        .group_by("datetime").mean()
        .drop("county")
    )
    
    df_historical_local = (
        df_historical
        .filter(pl.col("county").is_not_null())
        .group_by("county", "datetime").mean()
    )
    # Объединение всех обработанных данных с основным датафреймом df_data
    df_data = (
        df_data
        .join(df_gas, on="date", how="left")
        .join(df_client, on=["county", "is_business", "product_type", "date"], how="left")
        .join(df_electricity, on="datetime", how="left")
        
        .join(df_forecast_date, on="datetime", how="left", suffix="_fd")
        .join(df_forecast_local, on=["county", "datetime"], how="left", suffix="_fl")
        .join(df_historical_date, on="datetime", how="left", suffix="_hd")
        .join(df_historical_local, on=["county", "datetime"], how="left", suffix="_hl")
        
        .join(df_forecast_date.with_columns(pl.col("datetime") + pl.duration(days=7)), on="datetime", how="left", suffix="_fdw")
        .join(df_forecast_local.with_columns(pl.col("datetime") + pl.duration(days=7)), on=["county", "datetime"], how="left", suffix="_flw")
        .join(df_historical_date.with_columns(pl.col("datetime") + pl.duration(days=7)), on="datetime", how="left", suffix="_hdw")
        .join(df_historical_local.with_columns(pl.col("datetime") + pl.duration(days=7)), on=["county", "datetime"], how="left", suffix="_hlw")
        
        .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=2)).rename({"target": "target_1"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
        .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=3)).rename({"target": "target_2"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
        .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=4)).rename({"target": "target_3"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
        .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=5)).rename({"target": "target_4"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
        .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=6)).rename({"target": "target_5"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
        .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=7)).rename({"target": "target_6"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
        #.join(df_target.with_columns(pl.col("datetime") + pl.duration(days=14)).rename({"target": "target_7"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
        .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=8)).rename({"target": "target_7"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
        .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=9)).rename({"target": "target_8"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
        .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=10)).rename({"target": "target_9"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
        .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=11)).rename({"target": "target_10"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
        .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=12)).rename({"target": "target_11"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
        .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=13)).rename({"target": "target_12"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
        .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=14)).rename({"target": "target_13"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
        #.join(df_target.with_columns(pl.col("datetime") + pl.duration(days=335)).rename({"target": "target_y_1"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
        #.join(df_target.with_columns(pl.col("datetime") + pl.duration(days=340)).rename({"target": "target_y_2"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
        #.join(df_target.with_columns(pl.col("datetime") + pl.duration(days=365)).rename({"target": "target_y_3"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
        
        #Добавляем коолонку рабочий день или нет.
        .join(working_days, on="date", how="left")
        # Создание категориальных признаков и тригонометрических функций времени
        .with_columns(
            pl.col("datetime").dt.ordinal_day().alias("dayofyear"), # Добавление номера дня в году
            pl.col("datetime").dt.hour().alias("hour"),# Добавление часа
            pl.col("datetime").dt.day().alias("day"),# Добавление дня
            pl.col("datetime").dt.weekday().alias("weekday"),# Добавление дня недели
            pl.col("datetime").dt.month().alias("month"),# Добавление месяца
            pl.col("datetime").dt.year().alias("year"),# Добавление года
        )
        # Приведение типов данных
        .with_columns(
            pl.concat_str("county", "is_business", "product_type", "is_consumption", separator="_").alias("category_1"),
        )
        
        .with_columns(
            (np.pi * pl.col("dayofyear") / 183).sin().alias("sin(dayofyear)"), # Тригонометрические функции для дня в году
            (np.pi * pl.col("dayofyear") / 183).cos().alias("cos(dayofyear)"),
            (np.pi * pl.col("hour") / 12).sin().alias("sin(hour)"),
            (np.pi * pl.col("hour") / 12).cos().alias("cos(hour)"),
        )
        
        .with_columns(
            pl.col(pl.Float64).cast(pl.Float32),
        )
         # Удаление ненужных колонок
        .drop("date", "datetime", "hour", "dayofyear")
    )
    
    # return df_data, df_historical_local
    return df_data

In [ ]:
def to_pandas(X, y=None):
    cat_cols = ["county", "is_business", "product_type", "is_consumption", "category_1"]
    
    if y is not None:
        df = pd.concat([X.to_pandas(), y.to_pandas()], axis=1)
    else:
        df = X.to_pandas()    


    # Признаки из 12-го ноутбука Паши
    # ****************************************
    df = df.merge(df.groupby(['category_1', 'year', 'month', 'day'])['target_1'].sum().reset_index(),
                  on=['category_1', 'year', 'month', 'day'],
                  how='left',
                  suffixes=['', '_sum'])
    # ****************************************
    
    # Признаки из 14-го ноутбука Паши
    # ****************************************
    '''
    grouped_df = df.groupby(['category_1', 'year', 'month', 'day'])
    df = df.merge(grouped_df['target_1'].sum().reset_index(), on=['category_1', 'year', 'month', 'day'], how='left', suffixes=['', '_sum'])
    df = df.merge(grouped_df['target_1'].mean().reset_index(), on=['category_1', 'year', 'month', 'day'], how='left', suffixes=['', '_mean'])
    df = df.merge(grouped_df['target_6'].sum().reset_index(), on=['category_1', 'year', 'month', 'day'], how='left', suffixes=['', '_sum'])
    df = df.merge(grouped_df['target_6'].mean().reset_index(), on=['category_1', 'year', 'month', 'day'], how='left', suffixes=['', '_mean'])
    df = df.merge(grouped_df['target_13'].sum().reset_index(), on=['category_1', 'year', 'month', 'day'], how='left', suffixes=['', '_sum'])
    df = df.merge(grouped_df['target_13'].mean().reset_index(), on=['category_1', 'year', 'month', 'day'], how='left', suffixes=['', '_mean'])
    '''
    # ****************************************
    
    df = df.set_index("row_id")
    df[cat_cols] = df[cat_cols].astype("category")
    
    '''
    df["target_mean"] = df[[f"target_{i}" for i in range(1, 7)]].mean(1)
    df["target_std"] = df[[f"target_{i}" for i in range(1, 7)]].std(1)
    df["target_ratio"] = df["target_6"] / (df["target_7"] + 1e-3)
    '''
    '''
    df["target_mean_1"] = df[[f"target_{i}" for i in range(1, 13)]].mean(1)
    df["target_std_1"] = df[[f"target_{i}" for i in range(1, 13)]].std(1)
    df["target_mean_2"] = df[[f"target_{i}" for i in range(1, 6)]].mean(1)
    df["target_std_2"] = df[[f"target_{i}" for i in range(1, 6)]].std(1)
    df["target_ratio_1"] = df["target_1"] / (df["target_13"] + 1e-3)
    df["target_ratio_2"] = df["target_1"] / (df["target_6"] + 1e-3)
    df["target_ratio_3"] = df["target_6"] / (df["target_13"] + 1e-3)
    '''

    df["target_mean"] = df[[f"target_{i}" for i in range(1, 13)]].mean(1)
    df["target_std"] = df[[f"target_{i}" for i in range(1, 13)]].std(1)
    df["target_ratio"] = df["target_6"] / (df["target_13"] + 1e-3)
    '''
    df["target_ratio_y_1"] = df["target_y_1"] / (df["target_y_3"] + 1e-3)
    df["target_ratio_y_2"] = df["target_y_2"] / (df["target_y_3"] + 1e-3)
    '''
    
    return df

In [ ]:
root = "/kaggle/input/predict-energy-behavior-of-prosumers"

# Для локальных вычислений. Последний data_block_id тренировочной выборки
# А начиная со следующего data_block_id и до конца идет тест
train_end_data_block_id = 500
# train_end_data_block_id = 600

data_cols        = ['target', 'county', 'is_business', 'product_type', 'is_consumption', 'datetime', 'row_id', 'data_block_id']
# В df_data_cols колонки в таком порядке в каком они потом формируются в df_data
df_data_cols     = ['county', 'is_business', 'product_type', 'target', 'is_consumption', 'datetime', 'data_block_id', 'row_id']
client_cols      = ['product_type', 'county', 'eic_count', 'installed_capacity', 'is_business', 'date']
gas_cols         = ['forecast_date', 'lowest_price_per_mwh', 'highest_price_per_mwh']
electricity_cols = ['forecast_date', 'euros_per_mwh']
forecast_cols    = ['latitude', 'longitude', 'hours_ahead', 'temperature', 'dewpoint', 'cloudcover_high', 'cloudcover_low', 'cloudcover_mid', 'cloudcover_total', '10_metre_u_wind_component', '10_metre_v_wind_component', 'forecast_datetime', 'direct_solar_radiation', 'surface_solar_radiation_downwards', 'snowfall', 'total_precipitation']
historical_cols  = ['datetime', 'temperature', 'dewpoint', 'rain', 'snowfall', 'surface_pressure','cloudcover_total','cloudcover_low','cloudcover_mid','cloudcover_high','windspeed_10m','winddirection_10m','shortwave_radiation','direct_solar_radiation','diffuse_radiation','latitude','longitude']
location_cols    = ['longitude', 'latitude', 'county']
target_cols      = ['target', 'county', 'is_business', 'product_type', 'is_consumption', 'datetime']

save_path = None
load_path = None

In [ ]:
# для оптуны
# Диапазон гиперпараметров для модели обучающейся на данных is_consumption=1 
def lgb_objective_cons1(trial):
    params = {
        'device'            : trial.suggest_categorical('device', ['gpu']),
        'gpu_platform_id'   : trial.suggest_int('gpu_platform_id', 1, 1),
        'gpu_device_id'     : trial.suggest_int('gpu_device_id', 0, 0),
        'n_estimators'      : trial.suggest_int('n_estimators', 1000, 2000),
        'verbose'           : trial.suggest_int('verbose', -1, -1),
        'random_state'      : trial.suggest_int('random_state', 42, 42),
        'objective'         : trial.suggest_categorical('objective', ['l2']),
        'num_leaves'        : trial.suggest_int('num_leaves', 20, 50),
        'learning_rate'     : trial.suggest_float('learning_rate', 0.01, 0.1),
        'colsample_bytree'  : trial.suggest_float('colsample_bytree', 0.1, 1.0),
        'colsample_bynode'  : trial.suggest_float('colsample_bynode', 0.1, 1.0),
        #'reg_alpha'         : trial.suggest_float('reg_alpha', 1e-2, 20.0),
        'reg_alpha'         : trial.suggest_float('reg_alpha', 1e-2, 10.0),
        'reg_lambda'        : trial.suggest_float('reg_lambda', 1e-2, 20.0),
        'min_child_samples' : trial.suggest_int('min_child_samples', 4, 256),
        'max_depth'         : trial.suggest_int('max_depth', -1, -1),
        'max_bin'           : trial.suggest_int('max_bin', 32, 200),
    }
    
    model  = lgb.LGBMRegressor(**params)
    X      = df_train[df_train['is_consumption']==1].drop(columns=["target", "datetime"]).reset_index(drop=True)
    y      = df_train[df_train['is_consumption']==1]["target"].reset_index(drop=True)
    scores = cross_val_score(model, X, y, cv=cv, scoring='neg_mean_absolute_error')
    
    return -np.mean(scores)

# Диапазон гиперпараметров для модели обучающейся на данных is_consumption=0
def lgb_objective_cons0(trial):
    params = {
        'device'            : trial.suggest_categorical('device', ['gpu']),
        'gpu_platform_id'   : trial.suggest_int('gpu_platform_id', 1, 1),
        'gpu_device_id'     : trial.suggest_int('gpu_device_id', 0, 0),
        'n_estimators'      : trial.suggest_int('n_estimators', 1000, 2000),
        'verbose'           : trial.suggest_int('verbose', -1, -1),
        'random_state'      : trial.suggest_int('random_state', 42, 42),
        'objective'         : trial.suggest_categorical('objective', ['l2']),
        'num_leaves'        : trial.suggest_int('num_leaves', 20, 50),
        'learning_rate'     : trial.suggest_float('learning_rate', 0.01, 0.1),
        'colsample_bytree'  : trial.suggest_float('colsample_bytree', 0.1, 1.0),
        'colsample_bynode'  : trial.suggest_float('colsample_bynode', 0.1, 1.0),
        #'reg_alpha'         : trial.suggest_float('reg_alpha', 1e-2, 20.0),
        #'reg_lambda'        : trial.suggest_float('reg_lambda', 1e-2, 20.0),
        'reg_alpha'         : trial.suggest_float('reg_alpha', 1e-2, 10.0),
        'reg_lambda'        : trial.suggest_float('reg_lambda', 12, 20.0),
        'min_child_samples' : trial.suggest_int('min_child_samples', 4, 256),
        'max_depth'         : trial.suggest_int('max_depth', -1, -1),
        'max_bin'           : trial.suggest_int('max_bin', 32, 200),
    }
    
    model  = lgb.LGBMRegressor(**params)
    X      = df_train[df_train['is_consumption']==0].drop(columns=["target", "datetime"]).reset_index(drop=True)
    y      = df_train[df_train['is_consumption']==0]["target"].reset_index(drop=True)
    scores = cross_val_score(model, X, y, groups=groups, cv=cv, scoring='neg_mean_absolute_error')
    
    return -np.mean(scores)

## Global Variables

## Исследование

In [ ]:
if is_local:
    # Загрузка данных об энергопотреблении
    train = pd.read_csv(os.path.join(root, "train.csv"))
    
    # Создание сводной таблицы с средними значениями целевой переменной (target)
    # для каждой комбинации даты, округа, типа продукта, бизнеса и потребления
    pivot_train = train.pivot_table(
        index='datetime',
        columns=['county', 'product_type', 'is_business', 'is_consumption'],
        values='target',
        aggfunc='mean'
    )
    
    # Переименование колонок для удобства доступа и интерпретации
    pivot_train.columns = ['county{}_productType{}_isBusiness{}_isConsumption{}'.format(*col) for col in pivot_train.columns.values]
    pivot_train.index = pd.to_datetime(pivot_train.index)
    
    pivot_train

### 2023 год 

In [ ]:
if is_local:
    # Копирование сводной таблицы для визуализации
    df_plot = pivot_train.copy()
    
    # Нормализация данных для визуализации
    df_plot = (df_plot - df_plot.min()) / (df_plot.max() - df_plot.min())
    
    # Ресемплирование данных по дням и вычисление средних значений
    df_plot_resampled_D = df_plot.resample('D').mean()
    
    # Визуализация нормализованных данных с прозрачностью (alpha=0.1)
    df_plot_resampled_D.loc['2022-7':].plot(alpha=0.1, color='green', figsize=(18, 6), legend=False)


In [ ]:
if is_local:
    # Выбор колонок, соответствующих различным категориям потребления
    columns_consumption_0 = df_plot_resampled_D.columns[df_plot_resampled_D.columns.str.contains('isConsumption0')]
    columns_consumption_1 = df_plot_resampled_D.columns[df_plot_resampled_D.columns.str.contains('isConsumption1')]
    
    # Создание фигуры для визуализации
    plt.figure(figsize=(15, 6))
    
    # Создание пустых линий для легенды
    plt.plot([], color='red', label='is_Consumption = 1')  # Изменено на желтый цвет
    plt.plot([], color='black', label='is_Consumption = 0')   # Изменено на черный цвет
    
    # Отображение легенды
    plt.legend()
    
    # Визуализация данных для 'is_Consumption = 0' черным цветом
    for column in columns_consumption_0:
        df_plot_resampled_D.loc['2022-7':, column].plot(alpha=0.1, color='black', legend=False)  # Изменено на черный
    
    # Визуализация данных для 'is_Consumption = 1' желтым цветом
    for column in columns_consumption_1:
        df_plot_resampled_D.loc['2022-7':, column].plot(alpha=0.1, color='red', legend=False)  # Изменено на желтый
    
    # Отображение графика
    plt.show()




## Подготовка данных

### Запись тестовых и тренировочных csv файлов

In [ ]:
if is_local:
    # Если выполняем локально
    train_path = 'train'
    if not os.path.exists(train_path):
        # Создание каталога, если его нет
        os.makedirs(train_path)
    test_path = 'example_test_files'
    if not os.path.exists(test_path):
        # Создание каталога, если его нет
        os.makedirs(test_path)
else:
    # Если сабмит
    train_path = root
# Путь, куда запишем csv файлы для теста

In [ ]:
# Разделяет датафрейм на тренировочную и тестовую часть
# Возвращает часть датафрейма для тренировки, тестовую часть датафрейма записывает в каталог с тестами
def split_train_test(filename):
    df = pd.read_csv(os.path.join(root, filename))
    
    #Запишем часть данных для теста
    test_df = df[df["data_block_id"] > train_end_data_block_id]
    if (filename =="train.csv"):
        # Берем только те ячейки где target был не нулевым
        test_df = test_df[test_df["target"].notnull()]
        
    test_df.to_csv(os.path.join(test_path, filename), index=False)

    #Запишем часть данных для трейна
    train_df = df[df["data_block_id"] <= train_end_data_block_id]
    train_df.to_csv(os.path.join(train_path, filename), index=False)

# Доводим до ума тестовые таблицы чтобы они были точно такие как в реальном сабмите
def test_dfs_tune():
    # Делаем таблицу revealed_targets.csv
    df = pd.read_csv(os.path.join(root, "train.csv"))
    df = df[df["data_block_id"] > train_end_data_block_id - 2]
    df["data_block_id"] += 2
    df = df[df["target"].notnull()]
    df.to_csv(os.path.join(test_path, 'revealed_targets.csv'), index=False)
    
    # Делаем таблицу test.csv
    df = pd.read_csv(os.path.join(test_path, "train.csv"))
    df.rename(columns={'datetime': 'prediction_datetime'}, inplace=True)
    df.drop('target', axis=1, inplace=True)
    df['currently_scored'] = False
    df.to_csv(os.path.join(test_path, 'test.csv'), index=False)
    
    # Делаем таблицу sample_submission.csv
    selected_columns = ['row_id', 'data_block_id']
    df = df[selected_columns]
    df['target'] = 0
    df.to_csv(os.path.join(test_path, 'sample_submission.csv'), index=False)

# Сборка разделения файлов
def make_split():
    # csv файлы которые будем делить:
    csv_names = ["train.csv", "client.csv", "gas_prices.csv", "electricity_prices.csv", "forecast_weather.csv", "historical_weather.csv"]
    for csv_name in csv_names:
        split_train_test(csv_name)
    # Доделываем тестовые таблицы
    test_dfs_tune()

In [ ]:
%%time

# Создаем файлы csv c тренировочными и тестовыми таблицами
if is_local:
    # Пока отключил создание тестовых файлом. У меня локально они есть
    #make_split()
    pass

### Data I/O

In [ ]:
%%time
df_data        = pl.read_csv(os.path.join(train_path, "train.csv"), columns=data_cols, try_parse_dates=True)
df_client      = pl.read_csv(os.path.join(train_path, "client.csv"), columns=client_cols, try_parse_dates=True)
df_gas         = pl.read_csv(os.path.join(train_path, "gas_prices.csv"), columns=gas_cols, try_parse_dates=True)
df_electricity = pl.read_csv(os.path.join(train_path, "electricity_prices.csv"), columns=electricity_cols, try_parse_dates=True)
df_forecast    = pl.read_csv(os.path.join(train_path, "forecast_weather.csv"), columns=forecast_cols, try_parse_dates=True)
df_historical  = pl.read_csv(os.path.join(train_path, "historical_weather.csv"), columns=historical_cols, try_parse_dates=True)
#df_location    = pl.read_csv(os.path.join(root, "weather_station_to_county_mapping.csv"), columns=location_cols, try_parse_dates=True)
df_location    = pl.read_csv('/kaggle/input/locations/county_lon_lats.csv', columns=location_cols, try_parse_dates=True)
df_target      = df_data.select(target_cols)
working_days   = pl.read_csv('/kaggle/input/working-days/working_days.csv', try_parse_dates=True)

schema_data        = df_data.schema
schema_client      = df_client.schema
schema_gas         = df_gas.schema
schema_electricity = df_electricity.schema
schema_forecast    = df_forecast.schema
schema_historical  = df_historical.schema
schema_target      = df_target.schema

### HyperParam Optimization

In [ ]:
# Подготовка данных для поиска гиперпараметров
if is_tuning:
    X, y = df_data.drop("target"), df_data.select("target")
    X = feature_eng(X, df_client, df_gas, df_electricity, df_forecast, df_historical, df_location, df_target, working_days)
    df_train = to_pandas(X, y)

    # df_train = df_train[df_train["target"].notnull()].query(training_start_date)
pass

In [ ]:
# Подбор гиперпараметров для модели is_consumption=1
if is_tuning:
    study = optuna.create_study(direction='minimize', study_name='Regressor')
    study.optimize(lgb_objective_cons1, n_trials=100, show_progress_bar=True)
pass

In [ ]:
# Подбор гиперпараметров для модели is_consumption=0
if is_tuning:
    study = optuna.create_study(direction='minimize', study_name='Regressor')
    study.optimize(lgb_objective_cons0, n_trials=100, show_progress_bar=True)
pass

### Validation

In [ ]:
'''result = cross_validate(
    estimator=lgb.LGBMRegressor(**best_params, random_state=42),
    X=df_train.drop(columns=["target"]), 
    y=df_train["target"],
    scoring="neg_mean_absolute_error",
    cv=MonthlyKFold(1),
)

print(f"Fit Time(s): {result['fit_time'].mean():.3f}")
print(f"Score Time(s): {result['score_time'].mean():.3f}")
print(f"Error(MAE): {-result['test_score'].mean():.3f}")'''
pass

### Training

#### Параметры для LGB is_consumption=1

In [ ]:
'''
if is_gpu:
    p1={'device': 'gpu', 'n_estimators': 1469, 'verbose': -1, 'random_state': 42, 'objective': 'l2', 'num_leaves': 39, 'learning_rate': 0.08040990249470174, 'colsample_bytree': 0.8531849083888259, 'colsample_bynode': 0.9214154009431396, 'reg_alpha': 6.929103452856734, 'reg_lambda': 10.279437815903039, 'min_child_samples': 49, 'max_depth': -1, 'max_bin': 126}

    p2={'device': 'gpu', 'n_estimators': 1746, 'verbose': -1, 'random_state': 41, 'objective': 'l2', 'num_leaves': 41, 'learning_rate': 0.08796044370229837, 'colsample_bytree': 0.7546586583194814, 'colsample_bynode': 0.9994687314470938, 'reg_alpha': 6.1003717335294585, 'reg_lambda': 7.696527980193375, 'min_child_samples': 48, 'max_depth': -1, 'max_bin': 127}

    p3={'device': 'gpu', 'n_estimators': 1294, 'verbose': -1, 'random_state': 43, 'objective': 'l2', 'num_leaves': 33, 'learning_rate': 0.08737561850018681, 'colsample_bytree': 0.8699017760704414, 'colsample_bynode': 0.9978274134547724, 'reg_alpha': 5.898515471116359, 'reg_lambda': 17.17473296794324, 'min_child_samples': 6, 'max_depth': -1, 'max_bin': 131}

    p4={'device': 'gpu', 'n_estimators': 1192, 'verbose': -1, 'random_state': 44, 'objective': 'l2', 'num_leaves': 38, 'learning_rate': 0.0872521925642551, 'colsample_bytree': 0.721555382595741, 'colsample_bynode': 0.9995199827970517, 'reg_alpha': 5.841920140684635, 'reg_lambda': 15.39391905374745, 'min_child_samples': 40, 'max_depth': -1, 'max_bin': 98}

    p5={'device': 'gpu', 'n_estimators': 1235, 'verbose': -1, 'random_state': 45, 'objective': 'l2', 'num_leaves': 43, 'learning_rate': 0.0908952879560077, 'colsample_bytree': 0.7105327970235108, 'colsample_bynode': 0.6641934858040608, 'reg_alpha': 5.760171959868691, 'reg_lambda': 6.087301777183299, 'min_child_samples': 70, 'max_depth': -1, 'max_bin': 128}

    p6={'device': 'gpu', 'n_estimators': 1474, 'verbose': -1, 'random_state': 46, 'objective': 'l2', 'num_leaves': 31, 'learning_rate': 0.09206716453416329, 'colsample_bytree': 0.7435685123828456, 'colsample_bynode': 0.953901546054432, 'reg_alpha': 6.282388240444042, 'reg_lambda': 11.93184947089518, 'min_child_samples': 55, 'max_depth': -1, 'max_bin': 129}

    p7={'device': 'gpu', 'n_estimators': 1425, 'verbose': -1, 'random_state': 47, 'objective': 'l2', 'num_leaves': 29, 'learning_rate': 0.09761288166264291, 'colsample_bytree': 0.7780272286445504, 'colsample_bynode': 0.6312053815371705, 'reg_alpha': 9.269919639881412, 'reg_lambda': 13.182320657371639, 'min_child_samples': 16, 'max_depth': -1, 'max_bin': 153}
    
    p8={'device': 'gpu', 'verbose': -1, 'random_state' : 48, 'objective': 'l2', 'n_estimators': 1569, 'learning_rate': 0.0954350912359592, 'colsample_bytree': 0.5643173334694846, 'colsample_bynode': 0.8507963404594194, 'reg_alpha': 0.5815184353302838, 'reg_lambda': 19.577675580975, 'min_child_samples': 83, 'max_depth': 15, 'max_bin': 125}
    
    p9={'device': 'gpu', 'verbose': -1, 'random_state' : 49, 'objective': 'l2', 'n_estimators': 1655, 'learning_rate': 0.09749234861597421, 'colsample_bytree': 0.5878083872493073, 'colsample_bynode': 0.7476463591217907, 'reg_alpha': 0.6253329425224532, 'reg_lambda': 19.31104697178572, 'min_child_samples': 41, 'max_depth': 15, 'max_bin': 116}

    p10={'device': 'gpu', 'verbose': -1, 'random_state' : 50, 'objective': 'l2', 'n_estimators': 1702, 'learning_rate': 0.08956547486313553, 'colsample_bytree': 0.553841254939378, 'colsample_bynode': 0.7707977076873439, 'reg_alpha': 3.4503195735482803, 'reg_lambda': 17.09137108374253, 'min_child_samples': 46, 'max_depth': 16, 'max_bin': 123}

    p11={'device': 'gpu', 'verbose': -1, 'random_state' : 51, 'objective': 'l2', 'n_estimators': 1662, 'learning_rate': 0.09936096276241443, 'colsample_bytree': 0.5262782304338314, 'colsample_bynode': 0.6861424640373375, 'reg_alpha': 1.1434350179754031, 'reg_lambda': 18.238183112413253, 'min_child_samples': 42, 'max_depth': 16, 'max_bin': 118}

    p12={'device': 'gpu', 'verbose': -1, 'random_state' : 52, 'objective': 'l2', 'n_estimators': 1596, 'learning_rate': 0.09145526299958297, 'colsample_bytree': 0.5813223295308532, 'colsample_bynode': 0.8019876742272201, 'reg_alpha': 1.7506271227424264, 'reg_lambda': 19.97101707600914, 'min_child_samples': 45, 'max_depth': 16, 'max_bin': 129}

    p13={'device': 'gpu', 'verbose': -1, 'random_state' : 53, 'objective': 'l2', 'n_estimators': 1611, 'learning_rate': 0.09177637954991985, 'colsample_bytree': 0.5406014712979323, 'colsample_bynode': 0.7948709121454631, 'reg_alpha': 1.4162076884265264, 'reg_lambda': 19.18451476080634, 'min_child_samples': 39, 'max_depth': 15, 'max_bin': 128}

    p14={'device': 'gpu', 'verbose': -1, 'random_state' : 54, 'objective': 'l2', 'n_estimators': 1722, 'learning_rate': 0.09721924587634734, 'colsample_bytree': 0.581822849292829, 'colsample_bynode': 0.8475502365142857, 'reg_alpha': 0.5558295705320715, 'reg_lambda': 17.676848729848402, 'min_child_samples': 76, 'max_depth': 15, 'max_bin': 121}

    p15={'device': 'gpu', 'verbose': -1, 'random_state': 42, 'objective': 'l2', 'n_estimators': 1953, 'learning_rate': 0.06292846025674187, 'colsample_bytree': 0.885312594375526, 'colsample_bynode': 0.7831589504736596, 'reg_alpha': 7.677890737474055, 'reg_lambda': 16.04294344413536, 'min_child_samples': 69, 'max_depth': 13, 'max_bin': 122}
    p16={'device': 'gpu', 'verbose': -1, 'random_state': 42, 'objective': 'l2', 'n_estimators': 1781, 'learning_rate': 0.054763617969941024, 'colsample_bytree': 0.9233976730768843, 'colsample_bynode': 0.8969396545503117, 'reg_alpha': 9.893144785987541, 'reg_lambda': 19.956571072303618, 'min_child_samples': 30, 'max_depth': 11, 'max_bin': 128}
    p17={'device': 'gpu', 'verbose': -1, 'random_state': 42, 'objective': 'l2', 'n_estimators': 1713, 'learning_rate': 0.053743689746779025, 'colsample_bytree': 0.8813612023144495, 'colsample_bynode': 0.8739181786588937, 'reg_alpha': 6.967728410825889, 'reg_lambda': 18.772923941576302, 'min_child_samples': 8, 'max_depth': 11, 'max_bin': 130}
    p18={'device': 'gpu', 'verbose': -1, 'random_state': 42, 'objective': 'l2', 'n_estimators': 1852, 'learning_rate': 0.0592383731526721, 'colsample_bytree': 0.9158803696144284, 'colsample_bynode': 0.8677708234107011, 'reg_alpha': 8.178421543960194, 'reg_lambda': 18.453388032680913, 'min_child_samples': 7, 'max_depth': 13, 'max_bin': 127}
    p19={'device': 'gpu', 'verbose': -1, 'random_state': 42, 'objective': 'l2', 'n_estimators': 1854, 'learning_rate': 0.06206288936616207, 'colsample_bytree': 0.8880309956381198, 'colsample_bynode': 0.8902254554150658, 'reg_alpha': 8.741757862603745, 'reg_lambda': 16.131881076970146, 'min_child_samples': 11, 'max_depth': 13, 'max_bin': 133}
    p20={'device': 'gpu', 'verbose': -1, 'random_state': 42, 'objective': 'l2', 'n_estimators': 1760, 'learning_rate': 0.05058624430058491, 'colsample_bytree': 0.8623393986024189, 'colsample_bynode': 0.9242352457544074, 'reg_alpha': 10.403491499807716, 'reg_lambda': 17.336966880450976, 'min_child_samples': 16, 'max_depth': 12, 'max_bin': 131}
    p21={'device': 'gpu', 'verbose': -1, 'random_state': 42, 'objective': 'l2', 'n_estimators': 1783, 'learning_rate': 0.053984137286090667, 'colsample_bytree': 0.7988556940541043, 'colsample_bynode': 0.8962066698736209, 'reg_alpha': 10.033954743421075, 'reg_lambda': 19.869585514651835, 'min_child_samples': 29, 'max_depth': 11, 'max_bin': 128}
    p22={'device': 'gpu', 'n_estimators': 1000,'verbose': -1,'objective': 'l2','learning_rate': 0.06258413085998576, 'colsample_bytree': 0.6527661140701613, 'colsample_bynode': 0.8106858631408332, 'lambda_l1': 5.065645378814257, 'lambda_l2': 9.81159370218779, 'min_data_in_leaf': 192, 'max_depth': 10, 'max_bin': 250}
    p23={'device': 'gpu', 'n_estimators': 1000,'verbose': -1,'objective': 'l2','learning_rate': 0.0632167263149817, 'colsample_bytree': 0.6958033941948067, 'colsample_bynode': 0.6030801666196094, 'lambda_l1': 7.137580620471935, 'lambda_l2': 9.348169401713742, 'min_data_in_leaf': 74, 'max_depth': 11, 'max_bin': 220}
    p24={'device': 'gpu', 'n_estimators': 1000,'verbose': -1,'objective': 'l2','learning_rate': 0.061236402165228264, 'colsample_bytree': 0.81427095118471, 'colsample_bynode': 0.6097376843527067, 'lambda_l1': 6.360490880385201, 'lambda_l2': 9.954136008333839, 'min_data_in_leaf': 238, 'max_depth': 13, 'max_bin': 180}
else:
    pass
'''
pass

#### Параметры для LGB is_consumption=0

In [ ]:
'''
if is_gpu:
    # Параметры для lgbm c GPU
    c1={'device': 'gpu', 'n_estimators': 1873, 'verbose': -1, 'random_state': 42, 'objective': 'l2', 'num_leaves': 49, 'learning_rate': 0.06628595395939138, 'colsample_bytree': 0.8413540714224521, 'colsample_bynode': 0.8666339129825499, 'reg_alpha': 9.255223781602002, 'reg_lambda': 16.025856453220072, 'min_child_samples': 124, 'max_depth': -1, 'max_bin': 36}

    c2={'device': 'gpu', 'n_estimators': 1706, 'verbose': -1, 'random_state': 41, 'objective': 'l2', 'num_leaves': 40, 'learning_rate': 0.0623359455604657, 'colsample_bytree': 0.9080615436043814, 'colsample_bynode': 0.9159548001426139, 'reg_alpha': 8.494878682040135, 'reg_lambda': 16.767807661924376, 'min_child_samples': 132, 'max_depth': -1, 'max_bin': 35}

    c3={'device': 'gpu', 'n_estimators': 1692, 'verbose': -1, 'random_state': 43, 'objective': 'l2', 'num_leaves': 50, 'learning_rate': 0.06467681942915654, 'colsample_bytree': 0.7276162343708921, 'colsample_bynode': 0.8958940747299091, 'reg_alpha': 9.481145856323984, 'reg_lambda': 16.29339982949364, 'min_child_samples': 151, 'max_depth': -1, 'max_bin': 43}

    c4={'device': 'gpu', 'n_estimators': 1739, 'verbose': -1, 'random_state': 44, 'objective': 'l2', 'num_leaves': 44, 'learning_rate': 0.06080249451894554, 'colsample_bytree': 0.9074942761725775, 'colsample_bynode': 0.9170081480548385, 'reg_alpha': 8.530769407103739, 'reg_lambda': 16.81737853958101, 'min_child_samples': 109, 'max_depth': -1, 'max_bin': 35}

    c5={'device': 'gpu', 'n_estimators': 1826, 'verbose': -1, 'random_state': 45, 'objective': 'l2', 'num_leaves': 49, 'learning_rate': 0.05467489957746241, 'colsample_bytree': 0.7682313698228109, 'colsample_bynode': 0.8873377406908334, 'reg_alpha': 9.966772322142026, 'reg_lambda': 16.04339228378297, 'min_child_samples': 129, 'max_depth': -1, 'max_bin': 32}

    c6={'device': 'gpu', 'n_estimators': 1735, 'verbose': -1, 'random_state': 46, 'objective': 'l2', 'num_leaves': 41, 'learning_rate': 0.060385171871545046, 'colsample_bytree': 0.9106107260853121, 'colsample_bynode': 0.9133013005189936, 'reg_alpha': 8.215835720465444, 'reg_lambda': 16.82269739762653, 'min_child_samples': 140, 'max_depth': -1, 'max_bin': 32}

    c7={'device': 'gpu', 'n_estimators': 1740, 'verbose': -1, 'random_state': 47, 'objective': 'l2', 'num_leaves': 41, 'learning_rate': 0.06813066410209488, 'colsample_bytree': 0.967948306909385, 'colsample_bynode': 0.9089768944050921, 'reg_alpha': 8.8593247133808, 'reg_lambda': 16.750261541012176, 'min_child_samples': 114, 'max_depth': -1, 'max_bin': 41}

    c8={'device': 'gpu', 'verbose': -1, 'random_state' : 48, 'objective': 'l2', 'n_estimators': 1961, 'learning_rate': 0.055041729559669385, 'colsample_bytree': 0.8054200071555299, 'colsample_bynode': 0.8827333010526346, 'reg_alpha': 9.649253442752036, 'reg_lambda': 16.98908601233005, 'min_child_samples': 54, 'max_depth': 12, 'max_bin': 36}

    c9={'device': 'gpu', 'verbose': -1, 'random_state' : 49, 'objective': 'l2', 'n_estimators': 1948, 'learning_rate': 0.06233594141892915, 'colsample_bytree': 0.8484245099171761, 'colsample_bynode': 0.899824429438312, 'reg_alpha': 10.7294451589117, 'reg_lambda': 17.69396992827211, 'min_child_samples': 45, 'max_depth': 13, 'max_bin': 32}

    c10={'device': 'gpu', 'verbose': -1, 'random_state' : 50, 'objective': 'l2', 'n_estimators': 1995, 'learning_rate': 0.0633853242094111, 'colsample_bytree': 0.9331894149297775, 'colsample_bynode': 0.972632605889707, 'reg_alpha': 8.983517796023829, 'reg_lambda': 18.03334867391121, 'min_child_samples': 33, 'max_depth': 11, 'max_bin': 48}

    c11={'device': 'gpu', 'verbose': -1, 'random_state' : 51, 'objective': 'l2', 'n_estimators': 1948, 'learning_rate': 0.04980982203453618, 'colsample_bytree': 0.7850607568025258, 'colsample_bynode': 0.9990467893228645, 'reg_alpha': 8.279232611894738, 'reg_lambda': 18.878856721521696, 'min_child_samples': 85, 'max_depth': 12, 'max_bin': 32}

    c12={'device': 'gpu', 'verbose': -1, 'random_state' : 52, 'objective': 'l2', 'n_estimators': 1958, 'learning_rate': 0.06331649649993518, 'colsample_bytree': 0.965107198312526, 'colsample_bynode': 0.9562601410444004, 'reg_alpha': 8.595100697458118, 'reg_lambda': 19.672841466470988, 'min_child_samples': 38, 'max_depth': 14, 'max_bin': 37}

    c13={'device': 'gpu', 'verbose': -1, 'random_state' : 53, 'objective': 'l2', 'n_estimators': 1667, 'learning_rate': 0.06761829944908236, 'colsample_bytree': 0.8491878204722972, 'colsample_bynode': 0.7943301198687824, 'reg_alpha': 12.053887346204482, 'reg_lambda': 17.21503593146002, 'min_child_samples': 48, 'max_depth': 12, 'max_bin': 60}

    c14={'device': 'gpu', 'verbose': -1, 'random_state' : 54, 'objective': 'l2', 'n_estimators': 1965, 'learning_rate': 0.06236463049661708, 'colsample_bytree': 0.9209979630548757, 'colsample_bynode': 0.9718755549743984, 'reg_alpha': 9.485421137815203, 'reg_lambda': 19.728001606564117, 'min_child_samples': 24, 'max_depth': 14, 'max_bin': 35}

    c15={'device': 'gpu', 'verbose': -1, 'random_state': 42, 'objective': 'l2', 'n_estimators': 1928, 'learning_rate': 0.07746737348328141, 'colsample_bytree': 0.78312363495825, 'colsample_bynode': 0.519093615242415, 'reg_alpha': 3.5475701282438967, 'reg_lambda': 17.88655785653258, 'min_child_samples': 20, 'max_depth': 17, 'max_bin': 37}
    c16={'device': 'gpu', 'verbose': -1, 'random_state': 42, 'objective': 'l2', 'n_estimators': 1881, 'learning_rate': 0.07864103296902626, 'colsample_bytree': 0.7910482238038328, 'colsample_bynode': 0.48895580052374654, 'reg_alpha': 7.375666493081315, 'reg_lambda': 15.347283246448637, 'min_child_samples': 15, 'max_depth': 15, 'max_bin': 41}
    c17={'device': 'gpu', 'verbose': -1, 'random_state': 42, 'objective': 'l2', 'n_estimators': 1959, 'learning_rate': 0.06577095971205155, 'colsample_bytree': 0.8648745524121579, 'colsample_bynode': 0.4208704631805802, 'reg_alpha': 7.334717669004279, 'reg_lambda': 19.53863883138691, 'min_child_samples': 17, 'max_depth': 15, 'max_bin': 37}
    c18={'device': 'gpu', 'verbose': -1, 'random_state': 42, 'objective': 'l2', 'n_estimators': 1962, 'learning_rate': 0.06171792620856953, 'colsample_bytree': 0.9584819437986425, 'colsample_bynode': 0.5405402088572034, 'reg_alpha': 3.307191570022457, 'reg_lambda': 16.209300259712922, 'min_child_samples': 43, 'max_depth': 19, 'max_bin': 40}
    c19={'device': 'gpu', 'verbose': -1, 'random_state': 42, 'objective': 'l2', 'n_estimators': 1900, 'learning_rate': 0.07441152178622895, 'colsample_bytree': 0.8801531103073503, 'colsample_bynode': 0.337510749744266, 'reg_alpha': 6.105532634402442, 'reg_lambda': 19.489948983222703, 'min_child_samples': 28, 'max_depth': 11, 'max_bin': 51}
    c20={'device': 'gpu', 'verbose': -1, 'random_state': 42, 'objective': 'l2', 'n_estimators': 1923, 'learning_rate': 0.07413128975186019, 'colsample_bytree': 0.9108240472209401, 'colsample_bynode': 0.368498860839913, 'reg_alpha': 5.928691934278827, 'reg_lambda': 19.535434444939284, 'min_child_samples': 35, 'max_depth': 16, 'max_bin': 43}
    c21={'device': 'gpu', 'verbose': -1, 'random_state': 42, 'objective': 'l2', 'n_estimators': 1899, 'learning_rate': 0.07536122429341643, 'colsample_bytree': 0.8755415544495265, 'colsample_bynode': 0.3971515541568271, 'reg_alpha': 7.825234820629166, 'reg_lambda': 19.539558271427442, 'min_child_samples': 16, 'max_depth': 10, 'max_bin': 67}
    c22={'device': 'gpu', 'n_estimators': 1000,'verbose': -1,'objective': 'l2','learning_rate': 0.06258413085998576, 'colsample_bytree': 0.6527661140701613, 'colsample_bynode': 0.8106858631408332, 'lambda_l1': 5.065645378814257, 'lambda_l2': 9.81159370218779, 'min_data_in_leaf': 192, 'max_depth': 10, 'max_bin': 250}
    c23={'device': 'gpu', 'n_estimators': 1000,'verbose': -1,'objective': 'l2','learning_rate': 0.0632167263149817, 'colsample_bytree': 0.6958033941948067, 'colsample_bynode': 0.6030801666196094, 'lambda_l1': 7.137580620471935, 'lambda_l2': 9.348169401713742, 'min_data_in_leaf': 74, 'max_depth': 11, 'max_bin': 220}
    c24={'device': 'gpu', 'n_estimators': 1000,'verbose': -1,'objective': 'l2','learning_rate': 0.061236402165228264, 'colsample_bytree': 0.81427095118471, 'colsample_bynode': 0.6097376843527067, 'lambda_l1': 6.360490880385201, 'lambda_l2': 9.954136008333839, 'min_data_in_leaf': 238, 'max_depth': 13, 'max_bin': 180}
    
else:
    pass
'''
pass

#### Параметры для LGB is_consumption=1

In [ ]:
if is_gpu:
    p1={'device': 'gpu', 'verbose': -1, 'objective': 'l2', 'n_estimators': 1569, 'learning_rate': 0.0954350912359592, 'colsample_bytree': 0.5643173334694846, 'colsample_bynode': 0.8507963404594194, 'reg_alpha': 0.5815184353302838, 'reg_lambda': 19.577675580975, 'min_child_samples': 83, 'max_depth': 15, 'max_bin': 125}
    
    p2={'device': 'gpu', 'verbose': -1, 'objective': 'l2', 'n_estimators': 1655, 'learning_rate': 0.09749234861597421, 'colsample_bytree': 0.5878083872493073, 'colsample_bynode': 0.7476463591217907, 'reg_alpha': 0.6253329425224532, 'reg_lambda': 19.31104697178572, 'min_child_samples': 41, 'max_depth': 15, 'max_bin': 116}

    p3={'device': 'gpu', 'verbose': -1, 'objective': 'l2', 'n_estimators': 1702, 'learning_rate': 0.08956547486313553, 'colsample_bytree': 0.553841254939378, 'colsample_bynode': 0.7707977076873439, 'reg_alpha': 3.4503195735482803, 'reg_lambda': 17.09137108374253, 'min_child_samples': 46, 'max_depth': 16, 'max_bin': 123}

    p4={'device': 'gpu', 'verbose': -1, 'objective': 'l2', 'n_estimators': 1662, 'learning_rate': 0.09936096276241443, 'colsample_bytree': 0.5262782304338314, 'colsample_bynode': 0.6861424640373375, 'reg_alpha': 1.1434350179754031, 'reg_lambda': 18.238183112413253, 'min_child_samples': 42, 'max_depth': 16, 'max_bin': 118}

    p5={'device': 'gpu', 'verbose': -1, 'objective': 'l2', 'n_estimators': 1596, 'learning_rate': 0.09145526299958297, 'colsample_bytree': 0.5813223295308532, 'colsample_bynode': 0.8019876742272201, 'reg_alpha': 1.7506271227424264, 'reg_lambda': 19.97101707600914, 'min_child_samples': 45, 'max_depth': 16, 'max_bin': 129}

    p6={'device': 'gpu', 'verbose': -1, 'objective': 'l2', 'n_estimators': 1611, 'learning_rate': 0.09177637954991985, 'colsample_bytree': 0.5406014712979323, 'colsample_bynode': 0.7948709121454631, 'reg_alpha': 1.4162076884265264, 'reg_lambda': 19.18451476080634, 'min_child_samples': 39, 'max_depth': 15, 'max_bin': 128}

    p7={'device': 'gpu', 'verbose': -1, 'objective': 'l2', 'n_estimators': 1722, 'learning_rate': 0.09721924587634734, 'colsample_bytree': 0.581822849292829, 'colsample_bynode': 0.8475502365142857, 'reg_alpha': 0.5558295705320715, 'reg_lambda': 17.676848729848402, 'min_child_samples': 76, 'max_depth': 15, 'max_bin': 121}

    p8={'device': 'gpu', 'n_estimators': 1656, 'verbose': -1, 'random_state': 42, 'objective': 'l2', 'num_leaves': 41, 'learning_rate': 0.044516384496327305, 'colsample_bytree': 0.8375548988212456, 'colsample_bynode': 0.5667409811919698, 'reg_alpha': 3.1759770693392193, 'reg_lambda': 11.018602402122836, 'min_child_samples': 5, 'max_depth': -1, 'max_bin': 58}

    p9={'device': 'gpu', 'n_estimators': 1603, 'verbose': -1, 'random_state': 41, 'objective': 'l2', 'num_leaves': 41, 'learning_rate': 0.03818179150947824, 'colsample_bytree': 0.7488971539445326, 'colsample_bynode': 0.5610283223532974, 'reg_alpha': 3.4833384149996096, 'reg_lambda': 10.927987824861278, 'min_child_samples': 49, 'max_depth': -1, 'max_bin': 58}
else:
    pass

#### Параметры для LGB is_consumption=0

In [ ]:
if is_gpu:
    # Параметры для lgbm c GPU
    c1={'device': 'gpu', 'verbose': -1, 'objective': 'l2', 'n_estimators': 1961, 'learning_rate': 0.055041729559669385, 'colsample_bytree': 0.8054200071555299, 'colsample_bynode': 0.8827333010526346, 'reg_alpha': 9.649253442752036, 'reg_lambda': 16.98908601233005, 'min_child_samples': 54, 'max_depth': 12, 'max_bin': 36}

    c2={'device': 'gpu', 'verbose': -1, 'objective': 'l2', 'n_estimators': 1948, 'learning_rate': 0.06233594141892915, 'colsample_bytree': 0.8484245099171761, 'colsample_bynode': 0.899824429438312, 'reg_alpha': 10.7294451589117, 'reg_lambda': 17.69396992827211, 'min_child_samples': 45, 'max_depth': 13, 'max_bin': 32}

    c3={'device': 'gpu', 'verbose': -1, 'objective': 'l2', 'n_estimators': 1995, 'learning_rate': 0.0633853242094111, 'colsample_bytree': 0.9331894149297775, 'colsample_bynode': 0.972632605889707, 'reg_alpha': 8.983517796023829, 'reg_lambda': 18.03334867391121, 'min_child_samples': 33, 'max_depth': 11, 'max_bin': 48}

    c4={'device': 'gpu', 'verbose': -1, 'objective': 'l2', 'n_estimators': 1948, 'learning_rate': 0.04980982203453618, 'colsample_bytree': 0.7850607568025258, 'colsample_bynode': 0.9990467893228645, 'reg_alpha': 8.279232611894738, 'reg_lambda': 18.878856721521696, 'min_child_samples': 85, 'max_depth': 12, 'max_bin': 32}

    c5={'device': 'gpu', 'verbose': -1, 'objective': 'l2', 'n_estimators': 1958, 'learning_rate': 0.06331649649993518, 'colsample_bytree': 0.965107198312526, 'colsample_bynode': 0.9562601410444004, 'reg_alpha': 8.595100697458118, 'reg_lambda': 19.672841466470988, 'min_child_samples': 38, 'max_depth': 14, 'max_bin': 37}

    c6={'device': 'gpu', 'verbose': -1, 'objective': 'l2', 'n_estimators': 1667, 'learning_rate': 0.06761829944908236, 'colsample_bytree': 0.8491878204722972, 'colsample_bynode': 0.7943301198687824, 'reg_alpha': 12.053887346204482, 'reg_lambda': 17.21503593146002, 'min_child_samples': 48, 'max_depth': 12, 'max_bin': 60}

    c7={'device': 'gpu', 'verbose': -1, 'objective': 'l2', 'n_estimators': 1965, 'learning_rate': 0.06236463049661708, 'colsample_bytree': 0.9209979630548757, 'colsample_bynode': 0.9718755549743984, 'reg_alpha': 9.485421137815203, 'reg_lambda': 19.728001606564117, 'min_child_samples': 24, 'max_depth': 14, 'max_bin': 35}

    c8={'device': 'gpu', 'n_estimators': 1947, 'verbose': -1, 'random_state': 42, 'objective': 'l2', 'num_leaves': 54, 'learning_rate': 0.05723355282907335, 'colsample_bytree': 0.9475962639323839, 'colsample_bynode': 0.5979396404779531, 'reg_alpha': 0.907591058077434, 'reg_lambda': 12.818913905376233, 'min_child_samples': 34, 'max_depth': -1, 'max_bin': 33}

    c9={'device': 'gpu', 'n_estimators': 1920, 'verbose': -1, 'random_state': 41, 'objective': 'l2', 'num_leaves': 57, 'learning_rate': 0.05006410941730473, 'colsample_bytree': 0.9708958855792246, 'colsample_bynode': 0.5521647573988697, 'reg_alpha': 1.6279412387851688, 'reg_lambda': 12.723104735826002, 'min_child_samples': 99, 'max_depth': -1, 'max_bin': 34}

    c10={'device': 'gpu', 'n_estimators': 1942, 'verbose': -1, 'random_state': 43, 'objective': 'l2', 'num_leaves': 50, 'learning_rate': 0.057507379262121315, 'colsample_bytree': 0.9402195759121595, 'colsample_bynode': 0.46574576530217693, 'reg_alpha': 1.5086048112799886, 'reg_lambda': 14.086405703124512, 'min_child_samples': 50, 'max_depth': -1, 'max_bin': 34}
else:
    pass

#### Функция выделения интервалов

In [ ]:
# Выделяыет из данных указанные интервалы.
# И в каждом интервале выбирает случайно лишь указанную долю данных
# df_train - датафрейм для обработки
# is_consumption какие данные возвращать данные в разрезе is_consumption
# data_block_id_intervals какие данные возвращать данные в разрезе is_consumption
#   это массив масивов. В каждой строке описание периода
#   первая колонка на сколько data_block_id в конеце обучени отстоит от доступного конца данных
#   вторая колонка сколько data_block_id будет в периоде на котором обучаемся.
#   Третья колонка какую долю от данных оставдять (от 0 до 1)
#   Периодов (строк) может быть произвольное количество
# data_block_id_min минмимальный data_block_id который будет в возвращаемых данных
def get_train_intervals(df_train, is_consumption, data_block_id_intervals, data_block_id_min):
    max_block_id = df_train["data_block_id"].max()
    
    df_train_int = df_train[(
        #  выбираем только те данные которые больше data_block_id_min
        (df_train['data_block_id']>=data_block_id_min)
        #  выбираем только те данные для обучения по is_consumption на которых специализируетсмя модель
        &(df_train['is_consumption']==is_consumption)
        # Оставляем только notnull таргеты
        &(df_train["target"].notnull())
    )]

    ind = 0
    for cur_interval in data_block_id_intervals:
        # вырезаем очередной дата блок в указанных границах
        cur_data_block = df_train_int[(
            # До какого data_block_id учим первый блок
            (df_train_int['data_block_id']<=max_block_id-cur_interval[0])
            # C какого data_block_id учим первый блок
            &(df_train_int['data_block_id']>(max_block_id-cur_interval[0]-cur_interval[1]))
        )]
        # print('1. cur_data_block:', cur_data_block['data_block_id'].nunique())
        # print('1.1 cur_data_block:', cur_data_block.shape[0])
        # Берем только часть случайную часть данных из блока
        cur_data_block = cur_data_block.sample(frac = cur_interval[2])
        # print('2. cur_data_block:', cur_data_block['data_block_id'].nunique())
        # print('2.1 cur_data_block:', cur_data_block.shape[0])
        if ind == 0:
            final_train_df = cur_data_block.copy()
        else:
            final_train_df = pd.concat([final_train_df, cur_data_block], ignore_index=True)
        ind += 1
    return(final_train_df)

#### Функция обучения моделей с диска

In [ ]:
# Обучает модели и данные, записаные на диск
# и записывает обученные модели обратно на диск
# models_names - список имен моделей
# models_path - путь на диске где хроанятся модели и данные

def fit_models_from_disk(models_names, models_path):
    # Код процесса, который будет выполнятся параллельно
    process_code1 = '''
import time
import lightgbm as lgb
import pandas as pd
from joblib import dump
from joblib import load

model = lgb.LGBMRegressor()
while True:
    model_name = input()
    if model_name == 'exit':
        print(f"Finish")
        break
    
    # Загружаем модель
    #model = load(os.path.join(models_path, f'{model_name}-init.joblib'))
    # Загружаем трейн
    #X = pd.read_pickle(os.path.join(models_path, f'{model_name}-x.pkl'))
    # Загружаем таргет
    #y = pd.read_pickle(os.path.join(models_path, f'{model_name}-y.pkl'))
    
    # Обучаем модель
    #model.fit(X, y)
    
    #dump(model, os.path.join(models_path, f'{model_name}-trained.joblib'))
    
    print(f"Complete: {command}")
'''
    # Запуск дочернего процесса в котором будут тренироваться моджели
    process1 = subprocess.Popen(['python', '-c', process_code],
                                stdin=subprocess.PIPE, 
                                stdout=subprocess.PIPE, text=True)
    for model_name in models_names:

        print(p_time(), 'fit model:', model_name)
        print(p_time(),'Start1')
        # Отправка команда обучать модель дочернему процессу
        process1.stdin.write(f'{model_name}\n')
        process1.stdin.flush()
        print(p_time(),'Start2')
        # Ждем сообщения о завершенеии тренировки модели от дочернеего процесса
        response1 = process1.stdout.readline().strip()
        print(p_time(),'Start3')
        
        print(p_time(),f"Дочерний процесс: {response1.strip()}")
        
        time.sleep(0)  # Пауза для иллюстрации, вы можете удалить это в реальном приложении
        
    # Отправка команды завершения
    #response, _ = process.communicate(input='exit\n')
    process1.stdin.write('exit\n')
    process1.stdin.flush()
    
    # Ожидание завершения дочернего процесса
    print(p_time(),'Start4')
    process1.wait()
    print(p_time(),'Start5')
    
    print(p_time(), "Обучение моделей завершено")

In [ ]:
# Обучает модели и данные, записаные на диск
# и записывает обученные модели обратно на диск
# models_names - список имен моделей
# models_path - путь на диске где хроанятся модели и данные

def fit_models_from_disk1(models_names, models_path):
    # Код процесса, который будет выполнятся параллельно
    process_code = '''
import time
import lightgbm as lgb

model = lgb.LGBMRegressor()
while True:
    command = input()
    if command == 'exit':
        print(f"Дочерний процесс 2 завершил работу")
        break
    # Выполнение задачи
    time.sleep(1)
    print(f"Дочерний процесс 2 выполнил задачу: {command}")
'''
    # Запуск дочернего процесса в котором будут тренироваться моджели
    process1 = subprocess.Popen(['python', '-c', process_code],
                                stdin=subprocess.PIPE, 
                                stdout=subprocess.PIPE, text=True)
    for model_name in models_names:

        print(p_time(), 'fit model:', model_name)
        print(p_time(),'Start1')
        # Отправка команда обучать модель дочернему процессу
        process1.stdin.write(f'{model_name}\n')
        process1.stdin.flush()
        print(p_time(),'Start2')
        # Ждем сообщения о завершенеии тренировки модели от дочернеего процесса
        response1 = process1.stdout.readline().strip()
        print(p_time(),'Start3')
        
        print(p_time(),f"Дочерний процесс: {response1.strip()}")
        
    # Отправка команды завершения
    #response, _ = process.communicate(input='exit\n')
    process1.stdin.write('exit\n')
    process1.stdin.flush()
    
    # Ожидание завершения дочернего процесса
    print(p_time(),'Start4')
    process1.wait()
    print(p_time(),'Start5')
    
    print(p_time(), "Обучение моделей завершено")

#### Класс моделей

In [ ]:
#***

# Класс обертка для моделей. Хранит различные пареметры для обучения моделей.
# Например диапазоны данных на которых учить модель, как часто обучать заново и другие параметры
class Models:

    # Путь где хранятся модели, тренировочные и тестовые выборки для них 
    models_path = 'models'
    
    # Инициализирует параметры обучения
    # init_model - готовый объект модели для обучения
    # alphas_* - Массивы с коэффициентами, на которые домножают предсказания
    # alphas_1 - массив для моделей с предсказанием is_consumption == 1
    # alphas_0 - массив для моделей с предсказанием is_consumption == 0
    def __init__(self, alphas_1, alphas_0):
        # Инициализируем словари
        # Ключами во всех словарях будет имя модели
        
        # Словарь с моделями
        self.models =  dict()
        
        # Словарь с описанием периодов обучения модели
        # пока это матрица. из двух столбцов и двух строк в каждой строке описание периода
        # первая колонка на сколько data_block_id в конеце обучени отстоит от доступного конца данных
        # вторая колонка сколько data_block_id будет в периоде на котором обучаемся.
        # data_block_id могут быть эквивалентны дням, но могут и отличаться, если данные будут подавать блоками не равными дням
        # либо если будут разрывы в данных. Но они точно будут эквиваленты циклам предсказания
        self.data_block_id_intervals  = dict()

        # Словарь с указанием по какой минимальный data_block_id отрезать данные.
        # То есть меньше data_block_id_min не берем данные для обучения в любом случае, чтобы не было определено в data_block_id_intervals
        self.data_block_id_min  = dict()
        
        # Если 1, то модель предназначена для предсказания потребления электричества
        # Если 0, то модель предназначена для предсказания производства электричества
        self.is_consumption = dict()
        
        # Раз во сколько иттераций обучат модель
        self.learn_again_period = dict()
        
        # Смещение для начала обучения модели. Добавляется к номеру итерации сабмита.
        # Скажем если смещение 6 номер итерации 1, а обучаемся раз в чем итераций. То обучение будет в первуже итерацию сабмита.
        self.learn_again_offset = dict()
        
        # Время в секундах сколько заняло последнее обучение модели
        self.last_learn_time = dict()

        # Предсказания модели
        self.predictions = dict()
        
        # Коэффициент на который умножать предсказания модели
        self.predictions_alpha = dict()
        
        #Номер итерации. Обновляется при вызове метода fit
        self.itter_n = 0
        
        # Массивы с коэффициентами, на которые домножают предсказания
        # В нулевом индексе массива если устаревание 0 дней, в 1 если 1 день устарела и так далее
        # self.alphas = [41/240, 39/240]
        self.alphas_1 = alphas_1
        self.alphas_0 = alphas_0
        
        # Устанавливаем в True если будем возвращать сохраненные ранее предикты
        self.is_saved_predict = False

        if not os.path.exists(self.models_path):
            # Создание каталога, если его нет
            os.makedirs(self.models_path)
    
    # Добавляет еще одну модель
    # model_name - название модели
    # new_model - объект модели
    def add_model(self, model_name, new_model, is_consumption, data_block_id_intervals,
                  data_block_id_min, learn_again_period, learn_again_offset):
        
        self.models[model_name] = new_model
        self.is_consumption[model_name] = is_consumption
        self.data_block_id_intervals[model_name] = data_block_id_intervals
        self.data_block_id_min[model_name] = data_block_id_min
        self.learn_again_period[model_name] = learn_again_period
        self.learn_again_offset[model_name] = learn_again_offset
        self.last_learn_time[model_name] = 0
        self.predictions[model_name] = []
        self.predictions_alpha[model_name] = []

        # Сохранаяем начальную модель на диске
        dump(new_model, os.path.join(self.models_path, f'{model_name}-init.joblib'))
        
    # Обучает модель
    # model_name - название модели
    # df_train - датафрейм который содержит данные для обучения и целевой признак
    def fit_one_model(self, model_name, df_train):
        #print(p_time(), 'fit model:', model_name)
        '''
        self.models[model_name].fit(
            X=df_train.drop(columns=["target", "data_block_id"]),
            #X=df_train.drop(columns=["target"]),
            y=df_train["target"]
        )
        '''
        '''
        print(p_time(), 'fit1')
        df_train.drop(columns=["target", "data_block_id"]).to_csv(
            os.path.join(self.models_path, f'{model_name}-x.csv'), index=False
        )
        print(p_time(), 'fit2')
        df_train["target"].to_csv(
            os.path.join(self.models_path, f'{model_name}-y.csv'), index=False
        )
        '''
        #print(p_time(), 'fit3')

        df_train.drop(columns=["target", "data_block_id"]).to_pickle(
            os.path.join(self.models_path, f'{model_name}-x.pkl')
        )
        #print(p_time(), 'fit4')
        df_train["target"].to_pickle(
            os.path.join(self.models_path, f'{model_name}-y.pkl')
        )
        #print(p_time(), 'fit5')
    
    # Делает предсказания от отдельной модели.
    # model_name - название модели
    # X - признаки на которых нужно сделать предсказание
    def predict_one_model(self, model_name, X):
        if self.is_saved_predict:
            # Загружаем ранее сохраненный предикт
            print(p_time(), 'load predict model:', model_name)
            y = predictions[model_name][self.itter_n]
        else:
            # Делаем новый предикт
            print(p_time(), 'predict model:', model_name)
            model = load(os.path.join(self.models_path, f'{model_name}-trained.joblib'))
            y = (model
                 .predict(X.drop(columns=["data_block_id"])).clip(0)
                 #.predict(X).clip(0)
                )
            self.predictions[model_name].append(y)

        if (((self.itter_n + self.learn_again_offset[model_name])
             % self.learn_again_period[model_name]) == 0):
            # Если модель только что училась ставим коэффициент побольшще
            self.predictions_alpha[model_name] = (self.alphas_0[0]
                                                  if self.is_consumption[model_name] == 0
                                                  else self.alphas_1[0])
        else:
            # Если модель только что училась в прошлой итерации
            # ставим коэффициент побольшще
            self.predictions_alpha[model_name] = (self.alphas_0[1]
                                                  if self.is_consumption[model_name] == 0
                                                  else self.alphas_1[1])
            
        return(y)
    
    # Обучает все добавленные модели для которых наступило время их обучения
    # df_train - датафрейм который содержит данные для обучения и целевой признак
    # itter_n - номер иттерации в сабмите.
    # если itter_n равен <=0. Значит первоначальное обучение и обучаем всем модели
    def fit(self, df_train, itter_n):
        self.itter_n = itter_n
        if self.is_saved_predict:
            # Не обучаем модели, если в режиме сохраненных предсказаний
            return()
            
        max_block_id = df_train["data_block_id"].max()
        # Перебираем все модели
        for m_name in self.learn_again_period:
            # Либо сказали все модели учить
            if ((itter_n <= 0)
                # Либо учим если номер итераиции в сабмите плюс смещение делится на целое на период обучения
                or (((itter_n + self.learn_again_offset[m_name])
                     % self.learn_again_period[m_name]) == 0)):
                
                '''
                d_i = self.data_block_id_intervals[m_name]
                # Выделяем данные для обучения
                df_train_int = df_train[
                    # До какого data_block_id учим первый блок
                    (((df_train['data_block_id']<=max_block_id-d_i[0][0])
                     # C какого data_block_id учим первый блок
                     &(df_train['data_block_id']>(max_block_id-d_i[0][0]-d_i[0][1])))
                    # До какого data_block_id учим второй блок
                    |((df_train['data_block_id']<=max_block_id-d_i[1][0])
                      # С какого data_block_id учим второй блок
                      &(df_train['data_block_id']>(max_block_id-d_i[1][0]-d_i[1][1]))))
                #  выбираем только те данные которые больше data_block_id_min
                &(df_train['data_block_id']>=self.data_block_id_min[m_name])
                #  выбираем только те данные для обучения по is_consumption на которых специализируетсмя модель
                &(df_train['is_consumption']==self.is_consumption[m_name])
                # Оставляем только notnull таргеты
                &(df_train["target"].notnull())
                ]
                '''
                df_train_int = df_train[(
                    #  выбираем только те данные которые больше data_block_id_min
                    (df_train['data_block_id']>=self.data_block_id_min[m_name])
                    #  выбираем только те данные для обучения по is_consumption на которых специализируетсмя модель
                    &(df_train['is_consumption']==self.is_consumption[m_name])
                    # Оставляем только notnull таргеты
                    &(df_train["target"].notnull())
                )]

                ind = 0
                for cur_interval in self.data_block_id_intervals[m_name]:
                    # вырезаем очередной дата блок в указанных границах
                    cur_data_block = df_train_int[(
                        # До какого data_block_id учим первый блок
                        (df_train_int['data_block_id']<=max_block_id-cur_interval[0])
                        # C какого data_block_id учим первый блок
                        &(df_train_int['data_block_id']>(max_block_id-cur_interval[0]-cur_interval[1]))
                    )]
                    # print('1. cur_data_block:', cur_data_block['data_block_id'].nunique())
                    # print('1.1 cur_data_block:', cur_data_block.shape[0])
                    # Берем только часть случайную часть данных из блока
                    cur_data_block = cur_data_block.sample(frac = cur_interval[2])
                    # print('2. cur_data_block:', cur_data_block['data_block_id'].nunique())
                    # print('2.1 cur_data_block:', cur_data_block.shape[0])
                    if ind == 0:
                        final_train_df = cur_data_block.copy()
                    else:
                        final_train_df = pd.concat([final_train_df, cur_data_block], ignore_index=True)
                    ind += 1

                '''
                print('df_train_int:', df_train_int['data_block_id'].nunique())
                print('df_train_int:', df_train_int.shape[0])
                print('df_train:', df_train['data_block_id'].nunique())
                print('df_train:', df_train.shape[0])
                print('final_train_df:', final_train_df['data_block_id'].nunique())
                print('final_train_df:', final_train_df.shape[0])
                '''
                
                # Обучаем модель
                self.fit_one_model(m_name, final_train_df)
        # Обучаем модели и данные, записаные на диск
        fit_models_from_disk(
            models_names = list(self.models.keys()),
            models_path = self.models_path
        )
    
    # Делает предсказание всеми добавленными моделями и сводит их в одно предсказание
    def predict(self, X):
        # Создаем датафрейм для предсказаний
        # Первый столбец содержит информацию 'is_consumption'
        # Для каждой модели отдельный столбец с предсказаниями
        # В predict_df_0 будут предсказания для моделей с is_consumption == 0
        self.predict_df_0 = pd.DataFrame(X['is_consumption'])
        # В predict_df_1 будут предсказания для моделей с is_consumption == 1
        self.predict_df_1 = pd.DataFrame(X['is_consumption'])
        
        # Перебираем все модели
        for m_name in self.learn_again_period:
            # Делаем предсказание в соответвующий датафрейм в столбец модели
            if self.is_consumption[m_name] == 0:
                self.predict_df_0[m_name] = self.predict_one_model(m_name, X).clip(0)
                # Домножаем предсказание на коэффициент
                self.predict_df_0[m_name] = self.predict_df_0[m_name] * self.predictions_alpha[m_name]
            else:
                self.predict_df_1[m_name] = self.predict_one_model(m_name, X).clip(0)
                self.predict_df_1[m_name] = self.predict_df_1[m_name] * self.predictions_alpha[m_name]

        # Суммируем предсказания моделей раздельно по датафреймам
        self.predict_df_0['target'] = self.predict_df_0.iloc[:, 1:].sum(axis=1) #mean(axis=1)
        self.predict_df_1['target'] = self.predict_df_1.iloc[:, 1:].sum(axis=1) #mean(axis=1)
        
        #self.predict_df_0['target'] = self.predict_df_0.iloc[:, 1:].mean(axis=1)
        #self.predict_df_1['target'] = self.predict_df_1.iloc[:, 1:].mean(axis=1)
        
        # Сведение в одно предсказание потребления и производства электричества у просьюмеров
        predict_df = self.predict_df_1[['is_consumption', 'target']]

        predict_df.loc[predict_df['is_consumption']==0, 'target'] = self.predict_df_0.loc[self.predict_df_0['is_consumption']==0, 'target']
        
        return predict_df['target'].values

In [ ]:
# Вариант для шести моделей с одинаковыми коэффцициентами на
# предсказания для моделей обученный в этой итерации и в прошлой
#models = Models(alphas_0=[1/5, 1/5], alphas_1=[1/5, 1/5])
#models = Models(alphas_1=[1/6, 1/6], alphas_0=[1/5, 1/5])
models = Models(alphas_0=[1,1], alphas_1=[1,1])

#### Добавление моделей

##### Вариант 1

In [ ]:
# Описание параметров моделей
'''
models_list = [
    {'model_name': 'model-1', 'new_model': lgb.LGBMRegressor(**p1), 'learn_again_period': 8, 'learn_again_offset': 0, 'is_consumption': 1, 'data_block_id_intervals': [[0,1000],[0,0]], 'data_block_id_min': 122},
    {'model_name': 'model-2', 'new_model': lgb.LGBMRegressor(**p2), 'learn_again_period': 8, 'learn_again_offset': 1, 'is_consumption': 1, 'data_block_id_intervals': [[0,1000],[0,0]], 'data_block_id_min': 122},
    {'model_name': 'model-3', 'new_model': lgb.LGBMRegressor(**p3), 'learn_again_period': 8, 'learn_again_offset': 2, 'is_consumption': 1, 'data_block_id_intervals': [[0,1000],[0,0]], 'data_block_id_min': 122},
    {'model_name': 'model-4', 'new_model': lgb.LGBMRegressor(**p4), 'learn_again_period': 8, 'learn_again_offset': 3, 'is_consumption': 1, 'data_block_id_intervals': [[0,1000],[0,0]], 'data_block_id_min': 122},
    {'model_name': 'model-5', 'new_model': lgb.LGBMRegressor(**p5), 'learn_again_period': 8, 'learn_again_offset': 4, 'is_consumption': 1, 'data_block_id_intervals': [[0,1000],[0,0]], 'data_block_id_min': 122},
    {'model_name': 'model-6', 'new_model': lgb.LGBMRegressor(**p6), 'learn_again_period': 8, 'learn_again_offset': 5, 'is_consumption': 1, 'data_block_id_intervals': [[0,1000],[0,0]], 'data_block_id_min': 122},
    {'model_name': 'model-7', 'new_model': lgb.LGBMRegressor(**p7), 'learn_again_period': 8, 'learn_again_offset': 6, 'is_consumption': 1, 'data_block_id_intervals': [[0,1000],[0,0]], 'data_block_id_min': 122},
    {'model_name': 'model-8', 'new_model': lgb.LGBMRegressor(**p8), 'learn_again_period': 8, 'learn_again_offset': 7, 'is_consumption': 1, 'data_block_id_intervals': [[0,1000],[0,0]], 'data_block_id_min': 122},
    {'model_name': 'model-9', 'new_model': lgb.LGBMRegressor(**p9), 'learn_again_period': 8, 'learn_again_offset': 0, 'is_consumption': 1, 'data_block_id_intervals': [[0,1000],[0,0]], 'data_block_id_min': 122},
    {'model_name': 'model-10', 'new_model': lgb.LGBMRegressor(**p10), 'learn_again_period': 8, 'learn_again_offset': 1, 'is_consumption': 1, 'data_block_id_intervals': [[0,1000],[0,0]], 'data_block_id_min': 122},
    {'model_name': 'model-11', 'new_model': lgb.LGBMRegressor(**p11), 'learn_again_period': 8, 'learn_again_offset': 2, 'is_consumption': 1, 'data_block_id_intervals': [[0,1000],[0,0]], 'data_block_id_min': 122},
    {'model_name': 'model-12', 'new_model': lgb.LGBMRegressor(**p12), 'learn_again_period': 8, 'learn_again_offset': 3, 'is_consumption': 1, 'data_block_id_intervals': [[0,1000],[0,0]], 'data_block_id_min': 122},
    {'model_name': 'model-13', 'new_model': lgb.LGBMRegressor(**p13), 'learn_again_period': 8, 'learn_again_offset': 4, 'is_consumption': 1, 'data_block_id_intervals': [[0,1000],[0,0]], 'data_block_id_min': 122},
    {'model_name': 'model-14', 'new_model': lgb.LGBMRegressor(**p14), 'learn_again_period': 8, 'learn_again_offset': 5, 'is_consumption': 1, 'data_block_id_intervals': [[0,1000],[0,0]], 'data_block_id_min': 122},
    {'model_name': 'model-15', 'new_model': lgb.LGBMRegressor(**p15), 'learn_again_period': 8, 'learn_again_offset': 6, 'is_consumption': 1, 'data_block_id_intervals': [[0,1000],[0,0]], 'data_block_id_min': 122},
    {'model_name': 'model-16', 'new_model': lgb.LGBMRegressor(**p16), 'learn_again_period': 8, 'learn_again_offset': 7, 'is_consumption': 1, 'data_block_id_intervals': [[0,1000],[0,0]], 'data_block_id_min': 122},
    {'model_name': 'model-17', 'new_model': lgb.LGBMRegressor(**p17), 'learn_again_period': 8, 'learn_again_offset': 0, 'is_consumption': 1, 'data_block_id_intervals': [[0,1000],[0,0]], 'data_block_id_min': 122},
    {'model_name': 'model-18', 'new_model': lgb.LGBMRegressor(**p18), 'learn_again_period': 8, 'learn_again_offset': 1, 'is_consumption': 1, 'data_block_id_intervals': [[0,1000],[0,0]], 'data_block_id_min': 122},
    {'model_name': 'model-19', 'new_model': lgb.LGBMRegressor(**p19), 'learn_again_period': 8, 'learn_again_offset': 2, 'is_consumption': 1, 'data_block_id_intervals': [[0,1000],[0,0]], 'data_block_id_min': 122},
    {'model_name': 'model-20', 'new_model': lgb.LGBMRegressor(**p20), 'learn_again_period': 8, 'learn_again_offset': 3, 'is_consumption': 1, 'data_block_id_intervals': [[0,1000],[0,0]], 'data_block_id_min': 122},
    {'model_name': 'model-21', 'new_model': lgb.LGBMRegressor(**p21), 'learn_again_period': 8, 'learn_again_offset': 4, 'is_consumption': 1, 'data_block_id_intervals': [[0,1000],[0,0]], 'data_block_id_min': 122},
    {'model_name': 'model-22', 'new_model': lgb.LGBMRegressor(**p22), 'learn_again_period': 8, 'learn_again_offset': 5, 'is_consumption': 1, 'data_block_id_intervals': [[0,1000],[0,0]], 'data_block_id_min': 122},
    {'model_name': 'model-23', 'new_model': lgb.LGBMRegressor(**p23), 'learn_again_period': 8, 'learn_again_offset': 6, 'is_consumption': 1, 'data_block_id_intervals': [[0,1000],[0,0]], 'data_block_id_min': 122},
    {'model_name': 'model-24', 'new_model': lgb.LGBMRegressor(**p24), 'learn_again_period': 8, 'learn_again_offset': 7, 'is_consumption': 1, 'data_block_id_intervals': [[0,1000],[0,0]], 'data_block_id_min': 122},
    {'model_name': 'model-solar-1', 'new_model': lgb.LGBMRegressor(**c1), 'learn_again_period': 8, 'learn_again_offset': 0, 'is_consumption': 0, 'data_block_id_intervals': [[0,1000],[0,0]], 'data_block_id_min': 122},
    {'model_name': 'model-solar-2', 'new_model': lgb.LGBMRegressor(**c2), 'learn_again_period': 8, 'learn_again_offset': 1, 'is_consumption': 0, 'data_block_id_intervals': [[0,1000],[0,0]], 'data_block_id_min': 122},
    {'model_name': 'model-solar-3', 'new_model': lgb.LGBMRegressor(**c3), 'learn_again_period': 8, 'learn_again_offset': 2, 'is_consumption': 0, 'data_block_id_intervals': [[0,1000],[0,0]], 'data_block_id_min': 122},
    {'model_name': 'model-solar-4', 'new_model': lgb.LGBMRegressor(**c4), 'learn_again_period': 8, 'learn_again_offset': 3, 'is_consumption': 0, 'data_block_id_intervals': [[0,1000],[0,0]], 'data_block_id_min': 122},
    {'model_name': 'model-solar-5', 'new_model': lgb.LGBMRegressor(**c5), 'learn_again_period': 8, 'learn_again_offset': 4, 'is_consumption': 0, 'data_block_id_intervals': [[0,1000],[0,0]], 'data_block_id_min': 122},
    {'model_name': 'model-solar-6', 'new_model': lgb.LGBMRegressor(**c6), 'learn_again_period': 8, 'learn_again_offset': 5, 'is_consumption': 0, 'data_block_id_intervals': [[0,1000],[0,0]], 'data_block_id_min': 122},
    {'model_name': 'model-solar-7', 'new_model': lgb.LGBMRegressor(**c7), 'learn_again_period': 8, 'learn_again_offset': 6, 'is_consumption': 0, 'data_block_id_intervals': [[0,1000],[0,0]], 'data_block_id_min': 122},
    {'model_name': 'model-solar-8', 'new_model': lgb.LGBMRegressor(**c8), 'learn_again_period': 8, 'learn_again_offset': 7, 'is_consumption': 0, 'data_block_id_intervals': [[0,1000],[0,0]], 'data_block_id_min': 122},
    {'model_name': 'model-solar-9', 'new_model': lgb.LGBMRegressor(**c9), 'learn_again_period': 8, 'learn_again_offset': 0, 'is_consumption': 0, 'data_block_id_intervals': [[0,1000],[0,0]], 'data_block_id_min': 122},
    {'model_name': 'model-solar-10', 'new_model': lgb.LGBMRegressor(**c10), 'learn_again_period': 8, 'learn_again_offset': 1, 'is_consumption': 0, 'data_block_id_intervals': [[0,1000],[0,0]], 'data_block_id_min': 122},
    {'model_name': 'model-solar-11', 'new_model': lgb.LGBMRegressor(**c11), 'learn_again_period': 8, 'learn_again_offset': 2, 'is_consumption': 0, 'data_block_id_intervals': [[0,1000],[0,0]], 'data_block_id_min': 122},
    {'model_name': 'model-solar-12', 'new_model': lgb.LGBMRegressor(**c12), 'learn_again_period': 8, 'learn_again_offset': 3, 'is_consumption': 0, 'data_block_id_intervals': [[0,1000],[0,0]], 'data_block_id_min': 122},
    {'model_name': 'model-solar-13', 'new_model': lgb.LGBMRegressor(**c13), 'learn_again_period': 8, 'learn_again_offset': 4, 'is_consumption': 0, 'data_block_id_intervals': [[0,1000],[0,0]], 'data_block_id_min': 122},
    {'model_name': 'model-solar-14', 'new_model': lgb.LGBMRegressor(**c14), 'learn_again_period': 8, 'learn_again_offset': 5, 'is_consumption': 0, 'data_block_id_intervals': [[0,1000],[0,0]], 'data_block_id_min': 122},
    {'model_name': 'model-solar-15', 'new_model': lgb.LGBMRegressor(**c15), 'learn_again_period': 8, 'learn_again_offset': 6, 'is_consumption': 0, 'data_block_id_intervals': [[0,1000],[0,0]], 'data_block_id_min': 122},
    {'model_name': 'model-solar-16', 'new_model': lgb.LGBMRegressor(**c16), 'learn_again_period': 8, 'learn_again_offset': 7, 'is_consumption': 0, 'data_block_id_intervals': [[0,1000],[0,0]], 'data_block_id_min': 122},
    {'model_name': 'model-solar-17', 'new_model': lgb.LGBMRegressor(**c17), 'learn_again_period': 8, 'learn_again_offset': 0, 'is_consumption': 0, 'data_block_id_intervals': [[0,1000],[0,0]], 'data_block_id_min': 122},
    {'model_name': 'model-solar-18', 'new_model': lgb.LGBMRegressor(**c18), 'learn_again_period': 8, 'learn_again_offset': 1, 'is_consumption': 0, 'data_block_id_intervals': [[0,1000],[0,0]], 'data_block_id_min': 122},
    {'model_name': 'model-solar-19', 'new_model': lgb.LGBMRegressor(**c19), 'learn_again_period': 8, 'learn_again_offset': 2, 'is_consumption': 0, 'data_block_id_intervals': [[0,1000],[0,0]], 'data_block_id_min': 122},
    {'model_name': 'model-solar-20', 'new_model': lgb.LGBMRegressor(**c20), 'learn_again_period': 8, 'learn_again_offset': 3, 'is_consumption': 0, 'data_block_id_intervals': [[0,1000],[0,0]], 'data_block_id_min': 122},
    {'model_name': 'model-solar-21', 'new_model': lgb.LGBMRegressor(**c21), 'learn_again_period': 8, 'learn_again_offset': 4, 'is_consumption': 0, 'data_block_id_intervals': [[0,1000],[0,0]], 'data_block_id_min': 122},
    {'model_name': 'model-solar-22', 'new_model': lgb.LGBMRegressor(**c22), 'learn_again_period': 8, 'learn_again_offset': 5, 'is_consumption': 0, 'data_block_id_intervals': [[0,1000],[0,0]], 'data_block_id_min': 122},
    {'model_name': 'model-solar-23', 'new_model': lgb.LGBMRegressor(**c23), 'learn_again_period': 8, 'learn_again_offset': 6, 'is_consumption': 0, 'data_block_id_intervals': [[0,1000],[0,0]], 'data_block_id_min': 122},
    {'model_name': 'model-solar-24', 'new_model': lgb.LGBMRegressor(**c24), 'learn_again_period': 8, 'learn_again_offset': 7, 'is_consumption': 0, 'data_block_id_intervals': [[0,1000],[0,0]], 'data_block_id_min': 122},
]
'''
pass

##### Вариант 2

In [ ]:
'''
models_list = [
    {'model_name': 'model-1', 'new_model': lgb.LGBMRegressor(**p1), 'learn_again_period': 3, 'learn_again_offset': 0, 'is_consumption': 1, 'data_block_id_intervals': [[0,100,1],[100,100,0.5],[200,180,0.3],[380,1000,0.2]], 'data_block_id_min': 0},
    {'model_name': 'model-2', 'new_model': lgb.LGBMRegressor(**p2), 'learn_again_period': 3, 'learn_again_offset': 1, 'is_consumption': 1, 'data_block_id_intervals': [[0,100,1],[100,100,0.5],[200,180,0.3],[380,1000,0.2]], 'data_block_id_min': 0},
    {'model_name': 'model-3', 'new_model': lgb.LGBMRegressor(**p3), 'learn_again_period': 3, 'learn_again_offset': 2, 'is_consumption': 1, 'data_block_id_intervals': [[0,90,1],[100,110,0.4],[200,180,0.3],[380,1000,0.2]], 'data_block_id_min': 0},
    {'model_name': 'model-4', 'new_model': lgb.LGBMRegressor(**p4), 'learn_again_period': 3, 'learn_again_offset': 0, 'is_consumption': 1, 'data_block_id_intervals': [[0,100,1],[100,100,0.5],[200,180,0.3],[380,1000,0.2]], 'data_block_id_min': 0},
    {'model_name': 'model-5', 'new_model': lgb.LGBMRegressor(**p5), 'learn_again_period': 3, 'learn_again_offset': 1, 'is_consumption': 1, 'data_block_id_intervals': [[0,100,1],[100,100,0.5],[200,180,0.3],[380,1000,0.2]], 'data_block_id_min': 0},
    {'model_name': 'model-6', 'new_model': lgb.LGBMRegressor(**p6), 'learn_again_period': 3, 'learn_again_offset': 2, 'is_consumption': 1, 'data_block_id_intervals': [[0,100,1],[100,100,0.5],[200,180,0.3],[380,1000,0.2]], 'data_block_id_min': 0},
    {'model_name': 'model-7', 'new_model': lgb.LGBMRegressor(**p7), 'learn_again_period': 3, 'learn_again_offset': 0, 'is_consumption': 1, 'data_block_id_intervals': [[0,100,1],[100,100,0.5],[200,180,0.3],[380,1000,0.2]], 'data_block_id_min': 0},
    {'model_name': 'model-solar-1', 'new_model': lgb.LGBMRegressor(**c1), 'learn_again_period': 2, 'learn_again_offset': 0, 'is_consumption': 0, 'data_block_id_intervals': [[0,100,1],[100,100,0.5],[200,180,0.3],[380,1000,0.2]], 'data_block_id_min': 0},
    {'model_name': 'model-solar-2', 'new_model': lgb.LGBMRegressor(**c2), 'learn_again_period': 2, 'learn_again_offset': 0, 'is_consumption': 0, 'data_block_id_intervals': [[0,100,1],[100,100,0.5],[200,180,0.3],[380,1000,0.2]], 'data_block_id_min': 0},
    {'model_name': 'model-solar-3', 'new_model': lgb.LGBMRegressor(**c3), 'learn_again_period': 2, 'learn_again_offset': 0, 'is_consumption': 0, 'data_block_id_intervals': [[0,100,1],[100,100,0.5],[200,180,0.3],[380,1000,0.2]], 'data_block_id_min': 0},
    {'model_name': 'model-solar-4', 'new_model': lgb.LGBMRegressor(**c4), 'learn_again_period': 2, 'learn_again_offset': 0, 'is_consumption': 0, 'data_block_id_intervals': [[0,100,1],[100,100,0.5],[200,180,0.3],[380,1000,0.2]], 'data_block_id_min': 0},
    {'model_name': 'model-solar-5', 'new_model': lgb.LGBMRegressor(**c5), 'learn_again_period': 2, 'learn_again_offset': 0, 'is_consumption': 0, 'data_block_id_intervals': [[0,100,1],[100,100,0.5],[200,180,0.3],[380,1000,0.2]], 'data_block_id_min': 0},
    {'model_name': 'model-solar-6', 'new_model': lgb.LGBMRegressor(**c6), 'learn_again_period': 2, 'learn_again_offset': 1, 'is_consumption': 0, 'data_block_id_intervals': [[0,90,1],[100,110,0.4],[200,180,0.3],[380,1000,0.2]], 'data_block_id_min': 0},
    {'model_name': 'model-solar-7', 'new_model': lgb.LGBMRegressor(**c7), 'learn_again_period': 2, 'learn_again_offset': 1, 'is_consumption': 0, 'data_block_id_intervals': [[0,90,1],[100,110,0.4],[200,180,0.3],[380,1000,0.2]], 'data_block_id_min': 0},
    {'model_name': 'model-solar-8', 'new_model': lgb.LGBMRegressor(**c8), 'learn_again_period': 2, 'learn_again_offset': 1, 'is_consumption': 0, 'data_block_id_intervals': [[0,100,1],[100,100,0.5],[200,180,0.3],[380,1000,0.2]], 'data_block_id_min': 0},
]
'''

pass

##### Вариант 3

In [ ]:
'''
models_list = [
    {'model_name': 'model-1', 'new_model': lgb.LGBMRegressor(**p1), 'learn_again_period': 2, 'learn_again_offset': 0, 'is_consumption': 1, 'data_block_id_intervals': [[0,100,1],[100,100,0.5],[200,180,0.3],[380,1000,0.2]], 'data_block_id_min': 0},
    {'model_name': 'model-2', 'new_model': lgb.LGBMRegressor(**p2), 'learn_again_period': 2, 'learn_again_offset': 1, 'is_consumption': 1, 'data_block_id_intervals': [[0,100,1],[100,100,0.5],[200,180,0.3],[380,1000,0.2]], 'data_block_id_min': 0},
    {'model_name': 'model-3', 'new_model': lgb.LGBMRegressor(**p3), 'learn_again_period': 2, 'learn_again_offset': 0, 'is_consumption': 1, 'data_block_id_intervals': [[0,90,1],[100,110,0.4],[200,180,0.3],[380,1000,0.2]], 'data_block_id_min': 0},
    {'model_name': 'model-4', 'new_model': lgb.LGBMRegressor(**p4), 'learn_again_period': 2, 'learn_again_offset': 1, 'is_consumption': 1, 'data_block_id_intervals': [[0,100,1],[100,100,0.5],[200,180,0.3],[380,1000,0.2]], 'data_block_id_min': 0},
    {'model_name': 'model-5', 'new_model': lgb.LGBMRegressor(**p5), 'learn_again_period': 2, 'learn_again_offset': 0, 'is_consumption': 1, 'data_block_id_intervals': [[0,100,1],[100,100,0.5],[200,180,0.3],[380,1000,0.2]], 'data_block_id_min': 0},
    {'model_name': 'model-6', 'new_model': lgb.LGBMRegressor(**p6), 'learn_again_period': 2, 'learn_again_offset': 1, 'is_consumption': 1, 'data_block_id_intervals': [[0,100,1],[100,100,0.5],[200,180,0.3],[380,1000,0.2]], 'data_block_id_min': 0},
    {'model_name': 'model-solar-1', 'new_model': lgb.LGBMRegressor(**c1), 'learn_again_period': 1, 'learn_again_offset': 0, 'is_consumption': 0, 'data_block_id_intervals': [[0,100,1],[100,100,0.5],[200,180,0.3],[380,1000,0.2]], 'data_block_id_min': 0},
    {'model_name': 'model-solar-2', 'new_model': lgb.LGBMRegressor(**c2), 'learn_again_period': 1, 'learn_again_offset': 0, 'is_consumption': 0, 'data_block_id_intervals': [[0,100,1],[100,100,0.5],[200,180,0.3],[380,1000,0.2]], 'data_block_id_min': 0},
    {'model_name': 'model-solar-3', 'new_model': lgb.LGBMRegressor(**c3), 'learn_again_period': 1, 'learn_again_offset': 0, 'is_consumption': 0, 'data_block_id_intervals': [[0,100,1],[100,100,0.5],[200,180,0.3],[380,1000,0.2]], 'data_block_id_min': 0},
    {'model_name': 'model-solar-4', 'new_model': lgb.LGBMRegressor(**c4), 'learn_again_period': 1, 'learn_again_offset': 0, 'is_consumption': 0, 'data_block_id_intervals': [[0,100,1],[100,100,0.5],[200,180,0.3],[380,1000,0.2]], 'data_block_id_min': 0},
    {'model_name': 'model-solar-5', 'new_model': lgb.LGBMRegressor(**c5), 'learn_again_period': 1, 'learn_again_offset': 0, 'is_consumption': 0, 'data_block_id_intervals': [[0,100,1],[100,100,0.5],[200,180,0.3],[380,1000,0.2]], 'data_block_id_min': 0},
]
'''
pass

##### Вариант 4

In [ ]:
'''
models_list = [
    {'model_name': 'model-1', 'new_model': lgb.LGBMRegressor(**p1), 'learn_again_period': 1, 'learn_again_offset': 0, 'is_consumption': 1, 'data_block_id_intervals': [[0,90,1],[100,110,0.4],[200,180,0.3],[380,1000,0.2]], 'data_block_id_min': 0},
    {'model_name': 'model-2', 'new_model': lgb.LGBMRegressor(**p2), 'learn_again_period': 1, 'learn_again_offset': 0, 'is_consumption': 1, 'data_block_id_intervals': [[0,90,1],[100,110,0.4],[200,180,0.3],[380,1000,0.2]], 'data_block_id_min': 0},
    {'model_name': 'model-3', 'new_model': lgb.LGBMRegressor(**p3), 'learn_again_period': 1, 'learn_again_offset': 0, 'is_consumption': 1, 'data_block_id_intervals': [[0,90,1],[100,110,0.4],[200,180,0.3],[380,1000,0.2]], 'data_block_id_min': 0},
    {'model_name': 'model-4', 'new_model': lgb.LGBMRegressor(**p4), 'learn_again_period': 1, 'learn_again_offset': 0, 'is_consumption': 1, 'data_block_id_intervals': [[0,90,1],[100,110,0.4],[200,180,0.3],[380,1000,0.2]], 'data_block_id_min': 0},
    {'model_name': 'model-5', 'new_model': lgb.LGBMRegressor(**p5), 'learn_again_period': 1, 'learn_again_offset': 0, 'is_consumption': 1, 'data_block_id_intervals': [[0,90,1],[100,110,0.4],[200,180,0.3],[380,1000,0.2]], 'data_block_id_min': 0},
    {'model_name': 'model-6', 'new_model': lgb.LGBMRegressor(**p6), 'learn_again_period': 1, 'learn_again_offset': 0, 'is_consumption': 1, 'data_block_id_intervals': [[0,90,1],[100,110,0.4],[200,180,0.3],[380,1000,0.2]], 'data_block_id_min': 0},
    {'model_name': 'model-solar-1', 'new_model': lgb.LGBMRegressor(**c1), 'learn_again_period': 1, 'learn_again_offset': 0, 'is_consumption': 0, 'data_block_id_intervals': [[0,90,1],[100,110,0.4],[200,180,0.3],[380,1000,0.2]], 'data_block_id_min': 0},
    {'model_name': 'model-solar-2', 'new_model': lgb.LGBMRegressor(**c2), 'learn_again_period': 1, 'learn_again_offset': 0, 'is_consumption': 0, 'data_block_id_intervals': [[0,90,1],[100,110,0.4],[200,180,0.3],[380,1000,0.2]], 'data_block_id_min': 0},
    {'model_name': 'model-solar-3', 'new_model': lgb.LGBMRegressor(**c3), 'learn_again_period': 1, 'learn_again_offset': 0, 'is_consumption': 0, 'data_block_id_intervals': [[0,90,1],[100,110,0.4],[200,180,0.3],[380,1000,0.2]], 'data_block_id_min': 0},
    {'model_name': 'model-solar-4', 'new_model': lgb.LGBMRegressor(**c4), 'learn_again_period': 1, 'learn_again_offset': 0, 'is_consumption': 0, 'data_block_id_intervals': [[0,90,1],[100,110,0.4],[200,180,0.3],[380,1000,0.2]], 'data_block_id_min': 0},
    {'model_name': 'model-solar-5', 'new_model': lgb.LGBMRegressor(**c5), 'learn_again_period': 1, 'learn_again_offset': 0, 'is_consumption': 0, 'data_block_id_intervals': [[0,90,1],[100,110,0.4],[200,180,0.3],[380,1000,0.2]], 'data_block_id_min': 0},
    {'model_name': 'model-solar-6', 'new_model': lgb.LGBMRegressor(**c6), 'learn_again_period': 1, 'learn_again_offset': 0, 'is_consumption': 0, 'data_block_id_intervals': [[0,90,1],[100,110,0.4],[200,180,0.3],[380,1000,0.2]], 'data_block_id_min': 0},
]
'''
pass

##### Вариант 5

In [ ]:
models_list = [
    {'model_name': 'model-1', 'new_model': lgb.LGBMRegressor(**p1), 'learn_again_period': 1, 'learn_again_offset': 0, 'is_consumption': 1, 'data_block_id_intervals': [[0,100,1],[100,100,0.5],[200,180,0.3],[380,1000,0.2]], 'data_block_id_min': 0},
    {'model_name': 'model-solar-1', 'new_model': lgb.LGBMRegressor(**c1), 'learn_again_period': 1, 'learn_again_offset': 0, 'is_consumption': 0, 'data_block_id_intervals': [[0,100,1],[100,100,0.5],[200,180,0.3],[380,1000,0.2]], 'data_block_id_min': 0},
]

pass

In [ ]:
# Добавляем модели в объект класса моделей
for model_param in models_list:
    models.add_model(**model_param)

#### Обучение моделей

In [ ]:
#if is_local:
#    dump(model_solar, 'model_solar.joblib')
#    dump(model, 'model_lgbm.joblib')

Расскоментировать при необходимости загрузку ранее сохраненных моделей

In [ ]:
# Для загрузки локально
#model_solar = load('model_solar.joblib')
#model = load('model_lgbm.joblib')

# Для загрузки на kaggle
#model_solar = load('/kaggle/input/enefit/model_solar.joblib')
#model = load('/kaggle/input/enefit/model_lgbm.joblib')

## Prediction

In [ ]:
if is_local:
    # Если выполняем локально, а не сабмитим на кагл,
    # то выбираем другое имя для файла submission.csv.
    # Потому что в submission.csv записать прав нет и вылетает по ошибке
    submission_name = 'submission_loc.csv'
else:
    submission_name = 'submission.csv'

### Содержимое public_timeseries_testing_util.py

С необходимыми праками. Решил не импортировать его. а прямо тут. Так удобнее переносить на kaggle

In [ ]:
'''
An unlocked version of the timeseries API intended for testing alternate inputs.
Mirrors the production timeseries API in the crucial respects, but won't be as fast.

ONLY works afer the first three variables in MockAPI.__init__ are populated.
'''

from typing import Sequence, Tuple


class MockApi:
    def __init__(self):
        '''
        YOU MUST UPDATE THE FIRST THREE LINES of this method.
        They've been intentionally left in an invalid state.

        Variables to set:
            input_paths: a list of two or more paths to the csv files to be served
            group_id_column: the column that identifies which groups of rows the API should serve.
                A call to iter_test serves all rows of all dataframes with the current group ID value.
            export_group_id_column: if true, the dataframes iter_test serves will include the group_id_column values.
        '''
        self.input_paths: Sequence[str] = ['example_test_files/test.csv',
                                   'example_test_files/revealed_targets.csv', 
                                   'example_test_files/client.csv',
                                   'example_test_files/historical_weather.csv',
                                   'example_test_files/forecast_weather.csv',
                                   'example_test_files/electricity_prices.csv',
                                   'example_test_files/gas_prices.csv',
                                   'example_test_files/sample_submission.csv']
        self.group_id_column: str = 'data_block_id'
        self.export_group_id_column: bool = False
        # iter_test is only designed to support at least two dataframes, such as test and sample_submission
        assert len(self.input_paths) >= 2

        self._status = 'initialized'
        self.predictions = []

    def iter_test(self) -> Tuple[pd.DataFrame]:
        '''
        Loads all of the dataframes specified in self.input_paths,
        then yields all rows in those dataframes that equal the current self.group_id_column value.
        '''
        if self._status != 'initialized':

            raise Exception('WARNING: the real API can only iterate over `iter_test()` once.')

        dataframes = []
        for pth in self.input_paths:
            dataframes.append(pd.read_csv(pth, low_memory=False))
        group_order = dataframes[0][self.group_id_column].drop_duplicates().tolist()
        dataframes = [df.set_index(self.group_id_column) for df in dataframes]

        for group_id in group_order:
            self._status = 'prediction_needed'
            current_data = []
            for df in dataframes:
                cur_df = df.loc[group_id].copy()
                # returning single line dataframes from df.loc requires special handling
                if not isinstance(cur_df, pd.DataFrame):
                    cur_df = pd.DataFrame({a: b for a, b in zip(cur_df.index.values, cur_df.values)}, index=[group_id])
                    cur_df.index.name = self.group_id_column
                cur_df = cur_df.reset_index(drop=not(self.export_group_id_column))
                current_data.append(cur_df)
            yield tuple(current_data)

            while self._status != 'prediction_received':
                print('You must call `predict()` successfully before you can continue with `iter_test()`', flush=True)
                yield None

        with open(submission_name, 'w') as f_open:
            pd.concat(self.predictions).to_csv(f_open, index=False)
        self._status = 'finished'

    def predict(self, user_predictions: pd.DataFrame):
        '''
        Accepts and stores the user's predictions and unlocks iter_test once that is done
        '''
        if self._status == 'finished':
            raise Exception('You have already made predictions for the full test set.')
        if self._status != 'prediction_needed':
            raise Exception('You must get the next test sample from `iter_test()` first.')
        if not isinstance(user_predictions, pd.DataFrame):
            raise Exception('You must provide a DataFrame.')

        self.predictions.append(user_predictions)
        self._status = 'prediction_received'


def make_env():
    return MockApi()


### Функция для скора

In [ ]:
# Вычисляет скор для предсказаний, который были поданы на вход
# compare - датафрейм с уже заполенными реальными значениями таргета
# и сделанными предсказаниями

def compute_compare(compare, index_name):
    mae = mean_absolute_error(compare['target'] , compare['predict'])

    if (compare['data_block_id'].max() >= 518):
        compare_temp = compare[(compare['data_block_id'] >= 518)&(compare['data_block_id'] <= 606)]
        mae_518_606 = mean_absolute_error(compare_temp['target'], compare_temp['predict'])
    else:
        # Если еще не дошли до data_block_id >= 518 не считаем эти величины
        mae_518_606 = '-'
    
    if (compare['data_block_id'].max() > 600):
        # Считаем MAE для data_block_id > 600
        compare_temp = compare[compare['data_block_id'] > 600]
        mae_600 = mean_absolute_error(compare_temp['target'], compare_temp['predict'])
    else:
        # Если еще не дошли до data_block_id > 600 не считаем эти величины
        mae_600 = '-'

    mae_df = pd.DataFrame({
        '(ALL)': mae,
        'Feb - Apr (518 - 606)': mae_518_606,
        '(> 600)': mae_600
    }, index=[index_name])

    # Округляем числа до двух знаков после запятой и преобразуем их в строки
    mae_df = mae_df.round(3).astype(str)
    
    return mae_df

In [ ]:
# Подсчитывает скор.
# Возвращает датафрейм compare со сравнением предсказаний
def calc_score():
    # Загружаем предсказания
    #submission = pd.read_csv(submission_name)
    submission = pd.concat(env.predictions)
    
    # Загружаем истинные значения
    revealed_targets = pd.read_csv(os.path.join(test_path, "revealed_targets.csv"))
    revealed_targets['data_block_id'] -= 2
    revealed_targets = revealed_targets[revealed_targets["data_block_id"] > train_end_data_block_id]
    # Обрезаем реальные предсказания revealed_targets по длине уже сделанных предсказаний submission
    revealed_targets = revealed_targets.iloc[:len(submission)]

    # print(f'MAE: {mae}')
    
    # Подготовим данные для анализа изменения ошибки предсказания по мере удаления от времени завершения обучения
    compare = revealed_targets[['data_block_id', 'is_consumption', 'target']].copy()
    compare['predict'] = submission['target'].values
    compare['abs_err'] = abs(compare['predict'] - compare['target']).values
    compare['err'] = (compare['predict'] - compare['target']).values

    mae_df = compute_compare(compare, 'All Models MAE')

    new_mae_df = compute_compare(compare[compare['is_consumption']==0], 'is_consumption == 0')
    mae_df = pd.concat([mae_df, new_mae_df])

    new_mae_df = compute_compare(compare[compare['is_consumption']==1], 'is_consumption == 1')
    mae_df = pd.concat([mae_df, new_mae_df])

    # Перебираем предсказания всех моделей и складываем
    # скоры их предсказания в один датафрейм mae_df
    for model_name in models.predictions:
        # заполняем compareпредсказаниями модели
        compare['predict'] = np.concatenate(models.predictions[model_name])
        compare['abs_err'] = abs(compare['predict'] - compare['target']).values
        compare['err'] = (compare['predict'] - compare['target']).values
        # Оставляем в предсказаниях только те строки, которые предсказывала модель
        new_compare = compare[compare['is_consumption']==models.is_consumption[model_name]] 
        
        new_mae_df = compute_compare(new_compare,
                                     model_name+f' ({models.is_consumption[model_name]})')
                                                 
        mae_df = pd.concat([mae_df, new_mae_df])
    
    display(mae_df)
    print('MAE > 600', mae_df.loc['All Models MAE','(> 600)'])

    compare['predict'] = submission['target'].values
    compare['abs_err'] = abs(compare['predict'] - compare['target']).values
    compare['err'] = (compare['predict'] - compare['target']).values
    
    return compare

### Инициализация иттераций сабмита

In [ ]:
if is_local:
    # После этого можно имитировать локально загрузку при собмите на большом числе итераций
    # А не только четыре иттерации на 4 дня как в стандартной имитайии на кагле
    env = make_env()
else:
    # загружаем оригинальную библиотеку для сабмита
    import enefit
    env = enefit.make_env()

iter_test = env.iter_test()

### Цикл сабмита

In [ ]:
# Находим последний data_block_id в обучающих данных
max_train_data_block_id = df_data["data_block_id"].max()
# Устанавливаем первый data_block_id для теста следущим за тренировочным
cur_test_data_block_id = max_train_data_block_id + 1
cur_test_data_block_id

In [ ]:
%%time
count = 0

# Основной цикл для обработки данных тестового набора
for (test, revealed_targets, client, historical_weather,
        forecast_weather, electricity_prices, gas_prices, sample_prediction) in iter_test:
    iteration_start_time = time.time()
    print(p_time(), '*************** Iteration: ', count, '***************')
    
    # Переименование столбца для удобства
    test = test.rename(columns={"prediction_datetime": "datetime"})
    if is_local:
        # Если выполняем локально, то преобразуем некоторые типы данных
        # На кагле (а может и в линуксе) они и так преобразуются, но на виновс локально
        # не преобразуются и выдетают по ощибке
        test['datetime'] = pd.to_datetime(test['datetime'])
        client['date'] = pd.to_datetime(client['date'])
        gas_prices['origin_date'] = pd.to_datetime(gas_prices['origin_date'])
        gas_prices['forecast_date'] = pd.to_datetime(gas_prices['forecast_date'])
        electricity_prices['origin_date'] = pd.to_datetime(electricity_prices['origin_date'])
        electricity_prices['forecast_date'] = pd.to_datetime(electricity_prices['forecast_date'])
        forecast_weather['origin_datetime'] = pd.to_datetime(forecast_weather['origin_datetime'])
        forecast_weather['forecast_datetime'] = pd.to_datetime(forecast_weather['forecast_datetime'])
        historical_weather['datetime'] = pd.to_datetime(historical_weather['datetime'])
        revealed_targets['datetime'] = pd.to_datetime(revealed_targets['datetime'])
        
    # Добавляем колонку заполненную следующим data_block_id
    test["data_block_id"] = cur_test_data_block_id
    revealed_targets["data_block_id"] = cur_test_data_block_id
    
    df_test            = pl.from_pandas(test[data_cols[1:]], schema_overrides=schema_data)
    df_new_client      = pl.from_pandas(client[client_cols], schema_overrides=schema_client)
    df_new_gas         = pl.from_pandas(gas_prices[gas_cols], schema_overrides=schema_gas)
    df_new_electricity = pl.from_pandas(electricity_prices[electricity_cols], schema_overrides=schema_electricity)
    df_new_forecast    = pl.from_pandas(forecast_weather[forecast_cols], schema_overrides=schema_forecast)
    df_new_historical  = pl.from_pandas(historical_weather[historical_cols], schema_overrides=schema_historical)
    df_new_target      = pl.from_pandas(revealed_targets[target_cols], schema_overrides=schema_target)
    df_new_data        = pl.from_pandas(revealed_targets[df_data_cols], schema_overrides=schema_data)
    # Объединение новых данных с существующими и удаление дубликатов
    df_client          = pl.concat([df_client, df_new_client]).unique(subset=["county", "is_business", "product_type", "date"], maintain_order=True)
    df_gas             = pl.concat([df_gas, df_new_gas]).unique(subset=["forecast_date"], maintain_order=True)
    df_electricity     = pl.concat([df_electricity, df_new_electricity]).unique(subset=["forecast_date"], maintain_order=True)
    df_forecast        = pl.concat([df_forecast, df_new_forecast]).unique()
    df_historical      = pl.concat([df_historical, df_new_historical]).unique()
    df_target          = pl.concat([df_target, df_new_target]).unique()
    df_data            = pl.concat([df_data, df_new_data]).unique()
    
    
    #if (not(is_local) or (count == 0) or (count>=98)):
    if (not(is_local) or (count == 0) or (count>=17)):
    #if (not(is_local) or (count>=0)):
        # Если испольняем локально только после
        # итерации 100. Потому что интересует как ведет себя моделья через два месяца
        # после обучения. Так же обучаем все модели, если в самой первой итерции, если count == 0
        cur_time = time.time()
        if (((cur_time - notebook_starttime) < (8*60*60 + 60*50)) or is_disable_run_time_limit):
            # Не начинаем тренировать модели заново на реальном сабмите пока в датах предсказания не появятся даты идущие в скор
            if (df_test['datetime'].max() >= scor_start_time or (count == 0) or is_local):
                X, y = df_data.drop("target"), df_data.select("target")
                X = feature_eng(X, df_client, df_gas, df_electricity, df_forecast, df_historical, df_location, df_target, working_days)
                #Добавились новые строки в данные. Учим.
                df_train = to_pandas(X, y)
                models.fit(df_train=df_train, itter_n=count)
            else:
                print('Не тренеруем модель', df_test['datetime'].max(), 'не достигла даты начала тренировки:', scor_start_time)
                
        else:
            print('Не тренеруем модель, превышено время выполнения ноутбука:', (cur_time - notebook_starttime))

    # Применение функции инженерии признаков и преобразование данных обратно в pandas
    X_test = feature_eng(df_test, df_client, df_gas, df_electricity, df_forecast, df_historical, df_location, df_target, working_days)
    X_test = to_pandas(X_test)
    
    # Прогнозирование с использованием модели и ограничение предсказаний нулем
    test['target'] = models.predict(X_test)
    
    # Обновление целевых значений в примере предсказания
    sample_prediction["target"] = test['target']
    
    # Отправка предсказаний в среду выполнения
    env.predict(sample_prediction)

    if is_local:
        # Выводим текущий скор в разных разрезах
        compare = calc_score()
        pass
    
    count += 1
    # Переходим к следующему data_block_id на итерации тестов
    cur_test_data_block_id += 1
    print(p_time(), 'Iteration run time:', round(time.time() - iteration_start_time))
    print('')
    print('________________________________________________')
    print('')

## Анализ предсказания

### Подсчет скора

In [ ]:
if is_local:
    compare = calc_score()

In [ ]:
if is_local:
    dump(models.models['model-1'], 'models.joblib')
    pass

In [ ]:
models.models['model-1'].booster_.save_model('моя_модель.txt')

In [ ]:
#models = load('models.joblib')

In [ ]:
'''
count = 0
for cur_predict in models.predictions['model-1']:
    cur_predict = cur_predict * 0
    print(cur_predict)
    for model_name in models.predictions:
        #print(models.predictions[model_name][count])
        
        break
    count += 1
    
    break
'''

### График MAE по дням предсказания

In [ ]:
# выводит график средних ошибок сгруппированных по дням (точнее для блоков данных для предсказаний которые в целом эквиваленты дням)
def print_err(err_name, err_lable, err_title):
    # Группируем по data_block_id, то есть по дням и считаем отдельно для каждого дня предсказания MAE
    grouped_compare = compare.groupby('data_block_id').mean().reset_index()
    # Делаем скользящую среднюю
    grouped_compare['rolling_mean'] = grouped_compare[err_name].rolling(window=30, min_periods=1).mean()
    
    
    # Plotting the mean absolute errors
    plt.figure(figsize=(10, 8))
    #plt.bar(grouped_compare['data_block_id'], grouped_compare['abs_err'])
    plt.bar(grouped_compare['data_block_id'], grouped_compare[err_name], label=err_lable)
    plt.plot(grouped_compare['data_block_id'],
             grouped_compare['rolling_mean'],
             label='Rolling Mean (window=30)',
             color='orange',
             linestyle='-', linewidth=2)
    plt.xlabel('data_block_id')
    plt.ylabel(err_lable)
    plt.title(err_title)
    plt.legend()
    
    # Set ticks every 10 data_block_id
    tick_positions_y = np.arange(-40, max(grouped_compare[err_name]) + 1, 10)
    plt.yticks(tick_positions_y)
    plt.grid(True)
    plt.show()

In [ ]:
if is_local:
    print_err(err_name='abs_err',
              err_lable='Mean Absolute Error',
              err_title='Mean Absolute Error by data_block_id')

In [ ]:
if is_local:
    compare = compare[compare["data_block_id"] > 600]
    print_err(err_name='err',
              err_lable='Mean Error (predict - target)',
              err_title='Mean Error by data_block_id')